# Case-law Citation Helper via Model Merging

**Authors:** M. Fabiani, T. Falaschi, A. Franchini, R.A. Sacco

### Introduction
This project demonstrates an advanced workflow for building a specialized **Case-law Citation Helper** by synthesizing distinct legal competencies into a single LLM. Rather than relying on computationally expensive multi-task retraining, we leverage Parameter-Efficient Fine-Tuning **(PEFT)** and Model Merging techniques **(TIES/DARE)**. The goal is to create a model that excels at identifying correct legal holdings (via the CaseHOLD dataset) while maintaining robust, generalist legal reasoning across diverse tasks (via LegalBench). By merging two specialized LoRA experts, we would like to achieve a high-performance legal assistant that minimizes catastrophic forgetting and remains efficient for deployment.

### Project Roadmap
The notebook is structured into the following five key phases:

1.  **Environment Setup & Model Initialization**: Configuring the
environment with Unsloth for optimized 4-bit loading of the Llama 3.1 8B base model.

2. **Expert 1 Fine-Tuning (CaseHOLD)**: Training a LoRA adapter specialized in case-law precedent identification and holding selection.

3. **Expert 2 Fine-Tuning (LegalBench)**: Training a second LoRA adapter on a curated selection of 15+ tasks to capture broad legal logic and rule application.

4. **Model Merging (TIES/DARE)**: Merging the two adapters into a single set of weights using Mergekit, resolving parameter interference via sparsification and rescaling.

5. **Evaluation of Resulting Models**: Quantitatively comparing the merged model against the base model and individual experts to assess knowledge retention and synergy.

## Environment Setup & Model Initialization

Since legal text analysis and model fine-tuning are computationally intensive, we utilize **Unsloth**, a lightweight wrapper for the Hugging Face ecosystem that significantly accelerates training and reduces VRAM consumption.

 **Unsloth** provides up to 2x faster finetuning and consumes 30% less VRAM, allowing for the use of larger batch sizes without crashing the GPU. It achieves this efficiency by applying optimized "patches" to the model's layers and leveraging 4-bit quantization through bitsandbytes, which enables large-scale models to fit within standard memory constraints, such as those of an NVIDIA T4. By integrating memory-efficient attention mechanisms like xformers, Unsloth ensures a highly efficient workflow for Parameter-Efficient Fine-Tuning (PEFT) while maintaining the original model's accuracy

We install `unsloth` alongside essential libraries for Parameter-Efficient Fine-Tuning (PEFT). Key dependencies include:

- *bitsandbytes*: For 4-bit quantization, allowing the 8B parameter model to fit within standard GPU memory (e.g., NVIDIA T4 or L4).

- *trl & peft*: To handle the Supervised Fine-Tuning (SFT) and manage the Low-Rank Adaptation (LoRA) layers.

- *xformers*: For optimized memory-efficient attention mechanisms.

In [ ]:
# Install Unsloth and essential dependencies
!pip install unsloth
# Also install dependencies without overwriting the optimized Unsloth versions
!pip install --no-deps xformers trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/5

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
import torch
import os
import wandb
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback
import pandas as pd
from tqdm import tqdm
import re
import json
from IPython.display import display
import numpy as np
from transformers import TrainerCallback
import random
from datasets import Dataset
from collections import defaultdict
import contextlib
from peft import PeftModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# Setting global seed for reproducibility

GLOBAL_SEED = 42

def set_seed(seed: int = 42, deterministic: bool = True):

    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(GLOBAL_SEED, deterministic=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# PATH CONFIGURATION

BASE_PATH = "/content/drive/Shareddrives/MIRCV Project/LLM/"
model_casehold_path = BASE_PATH + "expert1_casehold"
path_expert_2 = BASE_PATH + "expert2_legalBench"


In [ ]:
def model_already_exists(path):
    """
    Checks if a valid LoRA adapter exists in the specified directory.
    We specifically look for 'adapter_config.json' and 'adapter_model.safetensors',
    which are the core files generated by PEFT/Unsloth.
    """
    config_path = os.path.join(path, "adapter_config.json")
    weights_path = os.path.join(path, "adapter_model.safetensors")

    # Check if both critical files exist
    return os.path.exists(config_path) and os.path.exists(weights_path)

In [ ]:
# check if expert1_caseHold exists
if model_already_exists(model_casehold_path):
    print(f"Found existing fine-tuned model at: {model_casehold_path}")
    print(" Skipping training phase. The existing model will be loaded.")
    RUN_TRAINING_caseHold = False
else:
    print(f"No existing model found at: {model_casehold_path}")
    print("Training phase will be executed.")
    RUN_TRAINING_caseHold = True

Found existing fine-tuned model at: /content/drive/Shareddrives/MIRCV Project/llm2/expert1_casehold
 Skipping training phase. The existing model will be loaded.


In [ ]:
# check if expert2_legalBench exists
if model_already_exists(path_expert_2):
    print(f"Found existing fine-tuned model at: {path_expert_2}")
    print("Skipping training phase. The existing model will be loaded.")
    RUN_TRAINING_legalBench = False
else:
    print(f"No existing model found at: {path_expert_2}")
    print("Training phase will be executed.")
    RUN_TRAINING_legalBench = True

Found existing fine-tuned model at: /content/drive/Shareddrives/MIRCV Project/llm2/expert2_legalBench
Skipping training phase. The existing model will be loaded.


### Initializing the Base Model (Llama 3.1 8B)

We selected **Meta-Llama-3.1-8B-Instruct** as the foundational model for this project due to its state-of-the-art performance within the compact 8-billion parameter class. This architecture represents the optimal trade-off for our hardware constraints: it is sufficiently powerful to handle complex legal reasoning while remaining lightweight enough to be fine-tuned on a standard Google Colab GPU (Tesla T4) using 4-bit quantization. We prioritized the **Instruct** variant over the base model because our downstream tasks, specifically CaseHOLD and LegalBench, rely heavily on precise instruction following and strict output formatting. Starting from an instruction-aligned checkpoint provides a superior baseline, allowing the model to focus on learning domain-specific legal nuances rather than relearning basic conversational structure.



We configure the `max_seq_length` to 1024 tokens. This choice represents a necessary strategic trade-off: it prevents "Out of Memory" (OOM) errors and significantly increases training speed, while remaining sufficient to encompass the critical context for the majority of the multiple-choice and classification tasks found in CaseHOLD and LegalBench.

A critical adjustment introduced here is setting the `tokenizer.truncation_side` to "left". This strategy is vital for both CaseHOLD and LegalBench: despite their differences, both datasets typically structure prompts with a lengthy legal text followed by the specific question or instruction at the very end. Default truncation (right-side) would disastrously cut off this final query. By enforcing left-side truncation, we guarantee that the model always retains the actual task instruction and answer options, sacrificing only the introductory context if the 1024-token limit is exceeded.

**Inference Setup**

We use Unsloth's `FastLanguageModel.for_inference(model)` utility, which optimizes the KV cache for 2x faster generation.

In [ ]:
# Configuration parameters
max_seq_length = 1024
dtype = None # Auto-detection (Float16 for Tesla T4, Bfloat16 for Ampere+)
load_in_4bit = True # Use 4-bit quantization to save VRAM
model_base = "unsloth/meta-llama-3.1-8b-instruct-bnb-4bit"
device_map = {"": 0}

In [ ]:
def upload_model(model_path):
  # Load the base model and tokenizer
  model, tokenizer = FastLanguageModel.from_pretrained(
      model_name = model_path,
      max_seq_length = max_seq_length,
      dtype = dtype,
      load_in_4bit = load_in_4bit,
      device_map = device_map,
      disable_log_stats = True
  )
  # This line ensures that if the text exceeds max_seq_length,
  # we truncate the beginning (the premises) but save the Options at the end.
  tokenizer.truncation_side = "left"
  return model, tokenizer

def load_base_with_lora_adapter(adapter_dir: str):
  model, tokenizer = upload_model(adapter_dir)
  FastLanguageModel.for_inference(model)
  return model, tokenizer

### LoRA Configuration for Future Merging

#### Low-Rank Adaptation (LoRA)

To efficiently adapt a pretrained LLM to legal tasks, we apply **Low-Rank Adaptation (LoRA)**, a parameter-efficient fine-tuning technique. Instead of updating the full weight matrix $W \in \mathbb{R}^{d \times d}$ of each linear layer, LoRA freezes the original pretrained weights and learns a lightweight update $\Delta W$ parameterized by two small low-rank matrices $A$ and $B$, where $A \in \mathbb{R}^{r \times d}$, $B \in \mathbb{R}^{d \times r}$, and $r \ll d$.

During training, the layer output becomes:

$$h = Wx + BAx = (W + \Delta W)x \quad \text{with} \quad \Delta W = BA$$

In this way, only the parameters of $A$ and $B$ are optimized via gradient descent, greatly reducing GPU memory usage and training cost.

After fine-tuning, the learned update can be merged into the base weights to obtain:

$$W_{\text{merged}} = W + BA$$

allowing inference with a standard linear layer while retaining the task-specific legal adaptation.

#### LoRA configuration

In our LoRA configuration, we utilize

- **Rank** ($r$) of 16: this value provides sufficient "expressive power" to capture the complex semantic patterns of legal language while ensuring the model remains lightweight and fits within the VRAM constraints of the Tesla T4.
- **Scaling factor** ($\alpha$) of 32: it stabilizes the influence of the LoRA adapters. It is set to $32$ to maintain a standard $2.0$ ratio ($\alpha/r$).
- targeting all available linear layers, including **attention** (q_proj, k_proj, v_proj, o_proj) and **MLP modules** (gate_proj, up_proj, down_proj), rather than just the attention heads.

Research indicates that spreading the trainable parameters across all projection matrices leads to higher quality adaptations and better performance on complex reasoning tasks. For legal domain-specific datasets like CaseHOLD and LegalBench, where deep semantic understanding of contractual and judicial language is required, this approach allows the model to capture more nuanced transformations within both the attention mechanism and the feed-forward networks, resulting in the significant performance gains observed.

Furthermore, by leveraging **Unsloth’s optimized gradient checkpointing,** we reduce VRAM usage by 30%, enabling efficient training on limited hardware like the Tesla T4 while fitting up to 2x larger batch sizes. This setup maintains the modularity needed to merge these specialized adapters later using the TIES method.


To enable TIES or DARE merging later, the LoRA (Low-Rank Adaptation) configuration must be identical for both legal experts.



In [ ]:
lora_rank = 16  # Rank of the adapter
lora_dropout = 0.05  # Optimized for Unsloth
lora_alpha = 32  # Alpha scaling factor

In [ ]:
def add_lora(model):
  # Add LoRA adapters to the model
  model = FastLanguageModel.get_peft_model(
      model,
      r = lora_rank,
      target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
      lora_alpha = lora_alpha,
      lora_dropout = lora_dropout,
      bias = "none",
      use_gradient_checkpointing = "unsloth", # "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
      random_state = 3407,
  )

  return model

### Utility function

In [ ]:
def clean_legal_text(text):
    """
    Cleans legal text by removing excessive whitespace and page references.
    """
    if not isinstance(text, str): return ""
    text = re.sub(r'\s+', ' ', text)           # Normalize whitespace/newlines
    text = re.sub(r'p{1,2}\.\s?\d+', '', text) # Remove page refs like 'p. 123'
    return text.strip()

In [ ]:
# class Callback
class DriveLogCallback(TrainerCallback):
    def __init__(self, output_file):
        self.output_file = output_file

    def on_log(self, args, state, control, logs=None, **kwargs):
        """
        This function is called every time the Trainer logs
        (each 'logging_steps' set in the arguments).
        """

        if state.is_local_process_zero:
            log_entry = logs.copy()
            log_entry['step'] = state.global_step
            log_entry['epoch'] = state.epoch

            # JSON Lines format
            with open(self.output_file, 'a') as f:
                f.write(json.dumps(log_entry) + "\n")

print("Callback ready to save logs to Drive!")

Callback ready to save logs to Drive!


In [ ]:
def print_training_logs(file_path):
  target_steps = [250, 500]

  print(f"{'Step':<10} | {'Training Loss':<15} | {'Validation Loss':<15}")
  print("-" * 45)

  results = {step: {"train": "N/A", "eval": "N/A"} for step in target_steps}

  with open(file_path, 'r') as f:
      for line in f:
          log = json.loads(line)
          step = log.get("step")

          if step in target_steps:
              if "loss" in log:
                  results[step]["train"] = f"{log['loss']:.6f}"
              if "eval_loss" in log:
                  results[step]["eval"] = f"{log['eval_loss']:.6f}"

  for step in target_steps:
      print(f"{step:<10} | {results[step]['train']:<15} | {results[step]['eval']:<15}")

## Expert 1 Fine-Tuning (CaseHOLD)

The objective of this phase is to develop our first "Legal Expert" specialized in the CaseHOLD task. This specific adapter focuses on identifying the correct legal "holding" (the core principle or finding of a court) within a given judicial context.

The **CaseHOLD dataset** (Case Holdings On Legal Decisions) provides 53,000+ multiple choice questions with prompts from a judicial decision and multiple potential holdings, one of which is correct, that could be cited.

**Holdings** are central to the common law system in the USA. They represent the the governing legal rule when the law is applied to a particular set of facts. It is what is precedential and what litigants can rely on in subsequent cases. The CaseHOLD task derived from the dataset is a ***multiple choice question answering task***, with five candidate holdings (one correct, four incorrect) for each citing context.

In [ ]:
if RUN_TRAINING_caseHold:
  # Load the base model and tokenizer
  model, tokenizer = upload_model(model_base)
  model = add_lora(model)
  print("Environment successfully initialized.")


else:
  print(f"Loading adapter from: {model_casehold_path}")
  # Load base model, then attach the LoRA adapter
  model, tokenizer = load_base_with_lora_adapter(model_casehold_path)

Loading adapter from: /content/drive/Shareddrives/MIRCV Project/llm2/expert1_casehold
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Unsloth 2026.1.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Data Preprocessing & Semantic Cleaning

Legal documents often contain "noise" from digitalization (e.g., erratic line breaks, page citations). Our preprocessing pipeline cleans the text and maps the five possible choices (0–4) to a structured multiple-choice format (A–E).


Given the extensive size of the original legal datasets, we have implemented a targeted subsetting strategy to ensure computational efficiency and prevent memory overflows during training. Rather than selecting samples at random, we utilize a custom stratified balancing function to create training, validation, and test sets. This process ensures that each label ($0$ through $4$) is represented by an equal number of examples, neutralizing any inherent class bias within the raw data. By maintaining a perfect class balance, we force the model to learn genuine legal reasoning across all possible outcomes, rather than simply exploiting the statistical frequency of more common labels.

In [ ]:
def formatting_prompts_func(examples):
    """
    Formats the raw dataset into Llama 3.1 Chat Template.
    Maps 0-4 labels to A-E options.
    """
    label_map = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}
    texts = []

    # Iterate through batch
    # 'endings' contains the list of 5 options
    for context, options, label in zip(examples["context"], examples["endings"], examples["label"]):

        # Apply cleaning
        clean_ctx = clean_legal_text(context)
        clean_opts = [clean_legal_text(opt) for opt in options]

        # Format options as a list A) ... B) ...
        formatted_options = "\n".join([f"{label_map[i]}) {opt}" for i, opt in enumerate(clean_opts)])

        # Construct the conversation for instruction tuning
        convo = [
            {"role": "system", "content": "You are a legal expert specializing in US case law."},
            {"role": "user", "content": f"CONTEXT: {clean_ctx}\n\nOPTIONS:\n{formatted_options}\n\nWhich option is the correct holding?"},
            {"role": "assistant", "content": f"The correct option is: {label_map[label]}"}
        ]

        # Apply the tokenizer's chat template
        # set add_generation_prompt=False for training (we want the model to learn the answer)
        texts.append(tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False))

    return { "text" : texts }

In [ ]:
# --- BALANCING FUNCTION ---
def get_balanced_subset(dataset, total_size, num_classes=5):
    """
    Creates a stratified/balanced subset by selecting an equal number
    of examples for each label (0, 1, 2, 3, 4).
    """
    samples_per_class = total_size // num_classes
    subsets = []

    # Iterate through each class label
    for label in range(num_classes):
        # Filter only examples belonging to the current class
        # We shuffle specific class data to ensure random selection
        class_subset = dataset.filter(lambda x: x["label"] == label).shuffle(seed=42)

        # Select the exact number of required samples
        # (Handles the edge case where a class might have fewer samples than requested)
        count = min(len(class_subset), samples_per_class)
        subsets.append(class_subset.select(range(count)))

    # Concatenate all class subsets and shuffle again
    # This prevents the dataset from being ordered by label (e.g., all As, then all Bs...)
    balanced_dataset = concatenate_datasets(subsets).shuffle(seed=42)
    return balanced_dataset

In [ ]:
from datasets import concatenate_datasets
from collections import Counter

train_size = 8000
validation_size = 500
test_size = 1000

# Load CaseHOLD
dataset_ch = load_dataset("coastalcph/lex_glue", "case_hold")

# Apply formatting function
dataset_ch = dataset_ch.map(formatting_prompts_func, batched=True)

train_dataset_ch = get_balanced_subset(dataset_ch["train"], train_size)
eval_dataset_ch = get_balanced_subset(dataset_ch["validation"], validation_size)
test_dataset_ch = get_balanced_subset(dataset_ch["test"], test_size)

def print_balance(name, ds):
    """Helper function to print the distribution of labels."""
    counts = Counter(ds["label"])
    print(f"\n{name} ({len(ds)} total):")
    for label, count in sorted(counts.items()):
        print(f"  Label {label}: {count} ({count/len(ds):.1%})")

# Print statistics to verify perfect balance
print_balance("Train Set", train_dataset_ch)
print_balance("Validation Set", eval_dataset_ch)
print_balance("Test Set", test_dataset_ch)

README.md: 0.00B [00:00, ?B/s]

case_hold/train-00000-of-00001.parquet:   0%|          | 0.00/40.5M [00:00<?, ?B/s]

case_hold/test-00000-of-00001.parquet:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

case_hold/validation-00000-of-00001.parq(…):   0%|          | 0.00/3.51M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/3600 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3600 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3600 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3600 [00:00<?, ? examples/s]


Train Set (8000 total):
  Label 0: 1600 (20.0%)
  Label 1: 1600 (20.0%)
  Label 2: 1600 (20.0%)
  Label 3: 1600 (20.0%)
  Label 4: 1600 (20.0%)

Validation Set (500 total):
  Label 0: 100 (20.0%)
  Label 1: 100 (20.0%)
  Label 2: 100 (20.0%)
  Label 3: 100 (20.0%)
  Label 4: 100 (20.0%)

Test Set (1000 total):
  Label 0: 200 (20.0%)
  Label 1: 200 (20.0%)
  Label 2: 200 (20.0%)
  Label 3: 200 (20.0%)
  Label 4: 200 (20.0%)


### Supervised Fine-Tuning (SFT) Strategy

The training configuration is designed to balance computational efficiency with the high precision required for legal domain adaptation on the CaseHold dataset.

- **Learning Rate and Stability:** A learning rate of $2e-5$ was chosen as it represents a "sweet spot" for fine-tuning large language models using LoRA. This relatively low value ensures that the model undergoes stable weight updates **without risking catastrophic forgetting**, which occurs when a model loses its pre-trained general knowledge by making overly aggressive adjustments. In a specialized field like law, maintaining the underlying linguistic logic while carefully adapting to specific legal terminology is crucial.

- **Learning Dynamics:** To further enhance stability, a **Cosine Learning Rate Scheduler** with a 10% warmup ratio is employed. This allows the model to start with a very low rate that gradually increases, preventing early training divergence, before smoothly decaying. This helps the model converge more effectively toward a global minimum during the single training epoch.

- **Memory Efficiency:** By setting the `per_device_train_batch_size` to 1 and `gradient_accumulation_steps` to 16, we achieve an effective batch size of 16. This allows for stable gradient updates while keeping the VRAM footprint low. The use of the adamw_8bit optimizer further reduces memory consumption by quantizing the optimizer states without significantly sacrificing precision.

- **Evaluation and Early Stopping:** To **prevent overfitting** on the legal corpus, an EarlyStoppingCallback is utilized with a patience of 2 evaluation cycles. Setting eval_strategy="steps" at **every 250 steps**, combined with load_best_model_at_end=True, ensures that the final model is the one that demonstrated the best generalization performance on the validation set.

In [ ]:
log_file_path_ch = BASE_PATH + "training_logs_caseHOLD.jsonl"
checkpoint_path = BASE_PATH + "checkpoints_caseHOLD"


In [ ]:
if RUN_TRAINING_caseHold:

  # Initialize Weights & Biases for logging
  run_name = "expert-1-casehold-lora"
  wandb.init(project="Legal-Model-Merging", name=run_name, mode="disabled")

  trainer = SFTTrainer(
      model=model,
      tokenizer=tokenizer,
      train_dataset=train_dataset_ch,
      eval_dataset=eval_dataset_ch,
      dataset_text_field="text",
      max_seq_length=max_seq_length,
      data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
      dataset_num_proc = 2,
      callbacks=[
        EarlyStoppingCallback(early_stopping_patience=2),
        DriveLogCallback(log_file_path_ch)
      ],

      args=TrainingArguments(
          per_device_train_batch_size = 1,
          gradient_accumulation_steps = 16,

          dataloader_num_workers = 2,     # Load data in parallel
          dataloader_pin_memory = True,   # Direct passage RAM -> VRAM

          warmup_ratio=0.1,
          num_train_epochs=1,
          learning_rate=2e-5,

          eval_strategy="steps",
          eval_steps=250,
          save_steps=250,

          fp16=not torch.cuda.is_bf16_supported(),
          bf16=torch.cuda.is_bf16_supported(),
          optim="adamw_8bit",
          lr_scheduler_type="cosine",
          output_dir=checkpoint_path,
          report_to="none",
          save_total_limit=1,
          load_best_model_at_end=True,
          logging_steps=10,
      ),
  )

In [ ]:
if RUN_TRAINING_caseHold:
  # Start Training
  print("Starting training...")
  trainer_stats = trainer.train()
else:
  print_training_logs(log_file_path_ch)

Step       | Training Loss   | Validation Loss
---------------------------------------------
250        | 1.632200        | 1.650124       
500        | 1.650800        | 1.634871       


### Saving the Expert Adapter

Upon completion, we export only the **LoRA adapters**. These small weight matrices (the "Expert") contain the specific knowledge acquired from CaseHOLD. Saving them separately allows us to reset the base model before training the second expert, ensuring no "knowledge leakage" occurs between the two training sessions, which is a prerequisite for a clean merge.

In [ ]:
if RUN_TRAINING_caseHold:
  # Save the adapter
  model.save_pretrained(model_casehold_path)
  tokenizer.save_pretrained(model_casehold_path)

  print(f"Expert 1 saved. Training time: {trainer_stats.metrics['train_runtime']:.2f} seconds.")

### Evaluation of Expert 1 (Inference & Accuracy)

To assess the effectiveness of our fine-tuning, we evaluated the model on a validation set comprising **500 samples**, which was kept separate from the training data. We measured the model's accuracy in selecting the correct legal holding (options A-E). The official test set was strictly reserved for the final evaluation phase to ensure an unbiased assessment of the completely merged model.

To ensure a rigorous and fair evaluation, the following methodology was applied:
1.  **Text Preprocessing:** Both the context and options were cleaned using the same pipeline applied during training to prevent data distribution shifts.
2.  **Strict Regex Matching:** We employed a Regular Expression-based evaluation metric to detect the correct answer option (A-E). This method filters out false positives (e.g., distinguishing the letter "A" from the word "Answer"), ensuring the reported accuracy reflects true model understanding.

**Evaluation Metric: Why Accuracy?**

For this experiment, **Accuracy** was selected as the primary performance metric. This choice is driven by the specific formulation of the **CaseHOLD** task, which is a multiple-choice question answering problem where the model must select the correct judicial holding from five distinct options (A-E).

Accuracy is the ideal metric for this scenario for two main reasons:
1.  **Class Balance:** The CaseHOLD dataset is relatively balanced across the correct answer labels, meaning accuracy provides a truthful representation of performance without the skewing effects seen in imbalanced datasets (where F1-score would be preferred).
2.  **Benchmark Standardization:** Accuracy is the official metric adopted by the **LexGLUE benchmark**, making our results directly comparable with the base Llama-3 model and other state-of-the-art legal language models reported in literature. It provides a clear, interpretable measure of the model's capability to distinguish correct legal reasoning from distractors.

This script loads the saved Fine-Tuned model from Google Drive and performs a
comparative evaluation against the Base model (Llama-3).
It calculates the accuracy on the CaseHOLD test set to quantify the
performance gain achieved through LoRA fine-tuning.
The script uses `disable_adapter()` to evaluate the base model without reloading weights.


In [ ]:
def evaluate_model(model, tokenizer, eval_data, use_adapter=True, using_test_set=True):

    correct = 0
    total = 0
    label_map = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}

    model.eval()
    with torch.no_grad():

      context_manager = model.disable_adapter() if not use_adapter else contextlib.nullcontext()
      # mode_name = "FINE-TUNED (Expert 1)" if use_adapter else "BASE (Llama-3)"

      set_name = "Test" if using_test_set else "Evaluation"
      print(f"\nStarting evaluation on {set_name} set...")

      with context_manager:
          for example in tqdm(eval_data):

              # We clean context and options exactly as in training
              clean_ctx = clean_legal_text(example["context"])
              clean_opts = [clean_legal_text(opt) for opt in example["endings"]]

              # PROMPT CREATION
              formatted_options = "\n".join([f"{label_map[i]}) {opt}" for i, opt in enumerate(clean_opts)])

              messages = [
                  {"role": "system", "content": "You are a legal expert specializing in US case law."},
                  {"role": "user", "content": f"CONTEXT: {clean_ctx}\n\nOPTIONS:\n{formatted_options}\n\nWhich option is the correct holding?"}
              ]

              try:
                  device = next(model.parameters()).device
              except StopIteration:
                  device = "cuda" if torch.cuda.is_available() else "cpu"

              inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)

              # GENERATION
              outputs = model.generate(inputs, max_new_tokens=20, temperature=0.01, pad_token_id=tokenizer.eos_token_id)
              prediction_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True).strip()

              # REGEX VERIFICATION (Anti-Bug)
              ground_truth = label_map[int(example["label"])] # E.g.: "A"

              # STRICT Regex: Looks for a single isolated letter A-E (word boundary)
              strict_pattern = r"\b([A-E])\b"

              # Search for the FIRST valid letter generated by the model
              match = re.search(strict_pattern, prediction_text)

              if match:
                  # Extract the letter chosen by the model (e.g., "B")
                  predicted_label = match.group(1)

                  # Direct comparison
                  if predicted_label == ground_truth:
                      correct += 1
                  # else: The model answered (e.g., "B") but it was wrong (expected "A") -> count remains same
              else:
                  # Case: The model did not write any A-E letter (e.g., "I don't know")
                  # Counted as an error.
                  pass

              total += 1

      accuracy = (correct / total) * 100
    return accuracy


In [ ]:
RESULTS_FILE_ch = BASE_PATH + "evaluation_results_caseHOLD.json"

# Flag to decide whether to perform evaluation or load existing results
PERFORM_EVALUATION_ch = not os.path.exists(RESULTS_FILE_ch)

In [ ]:
if PERFORM_EVALUATION_ch:
    # Evaluate Fine-Tuned Model
    acc_finetuned = evaluate_model(model, tokenizer, eval_dataset_ch, use_adapter=True, using_test_set=False)

    # Evaluate Base Model (Temporarily disabling LoRA adapters)
    acc_base = evaluate_model(model, tokenizer, eval_dataset_ch, use_adapter=False, using_test_set=False)
else:
    print(f"Skipping evaluation as results file already exists at: {RESULTS_FILE_ch}")
    # Initialize acc_finetuned and acc_base to None, as they won't be used for saving
    acc_finetuned = None
    acc_base = None

Skipping evaluation as results file already exists at: /content/drive/Shareddrives/MIRCV Project/llm2/evaluation_results_caseHOLD.json


In [ ]:
def save_results(base_acc, expert_acc, eval_size, results_file):
    """
    Saves the experiment results to a JSON file.
    """
    data = {
        "base_model_accuracy_caseHOLD": base_acc,
        "expert_model_accuracy_caseHOLD": expert_acc,
        "eval_samples_caseHOLD": eval_size
    }

    with open(results_file, 'w') as f:
        json.dump(data, f, indent=4)

    print(f" Results successfully saved to: {results_file}")


def load_and_display_results(results_file):
    """
    Loads results from the JSON file and displays a comparative table.
    If the file does not exist, it alerts the user.
    """
    if not os.path.exists(results_file):
        print(f" No results file found at {results_file}")
        return

    # Load data
    with open(results_file, 'r') as f:
        data = json.load(f)

    base_acc = data["base_model_accuracy_caseHOLD"]
    expert_acc = data["expert_model_accuracy_caseHOLD"]
    eval_size = data["eval_samples_caseHOLD"]

    # Calculate improvement
    improvement = expert_acc - base_acc

    # Create DataFrame for visualization
    results_df = pd.DataFrame({
        "Model": ["Llama-3.1-8B (Base)", "Expert 1 (Fine-Tuned)"],
        "Dataset": ["CaseHOLD", "CaseHOLD"],
        "Examples Used": [eval_size, eval_size],
        "Accuracy": [f"{base_acc:.2f}%", f"{expert_acc:.2f}%"],
        "Improvement": ["-", f" +{improvement:.2f}% "]
    })

    print("\n\n FINAL COMPARISON RESULTS:")
    display(results_df)

### Experimental Results: Base Model vs. Fine-Tuned Expert

The following table presents a comparative analysis between the base **Llama-3.1-8B** model and our **Fine-Tuned 'Expert 1'** adapter on the **CaseHOLD** evaluation set.


In [ ]:
if PERFORM_EVALUATION_ch:
    # Save the results to Drive
    save_results(acc_base, acc_finetuned, validation_size, RESULTS_FILE_ch)
    print("Results have been evaluated and saved. Displaying now:")
else:
    print("Results file already exists. Loading and displaying previous results:")

# Load and display the table (this will handle both cases)
load_and_display_results(RESULTS_FILE_ch)

Results file already exists. Loading and displaying previous results:


 FINAL COMPARISON RESULTS:


,Model,Dataset,Examples Used,Accuracy,Improvement
0,Llama-3.1-8B (Base),CaseHOLD,500,56.00%,-
1,Expert 1 (Fine-Tuned),CaseHOLD,500,74.80%,+18.80%




**Conclusion**

The quantitative results on the CaseHOLD validation set clearly demonstrate the efficacy of the domain-specific fine-tuning process.

 This remarkable improvement underscores the value of the PEFT (LoRA) approach for targeted specialization; it enables the model to bridge the gap between general linguistic understanding and the high-precision requirements of U.S. case law analysis, transforming it into a much more reliable tool for legal holding classification.

### Interactive Inference and Qualitative Analysis

We implemented an interactive inference pipeline. The `run_legal_chat` function facilitates a qualitative evaluation by simulating a real-world legal consultation scenario. This module randomly samples cases from the test set, processes them through the `format_prompt` utility to match the Llama-3 instruction template, and generates responses using greedy decoding (temperature=0.0) to ensure deterministic and precise outputs. This setup allows for a granular examination of the model's ability to interpret complex judicial contexts and select the correct holding among distractors, providing immediate visual feedback on its alignment with ground truth labels.

In [ ]:
def format_prompt(example, tokenizer, device):
    """
    Formats a single dataset example into the Llama 3 chat template.
    Returns the tokenized input tensors.
    """
    label_map = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}

    # Clean raw text
    clean_ctx = clean_legal_text(example["context"])
    clean_opts = [clean_legal_text(opt) for opt in example["endings"]]

    # Create the options list string (A, B, C...)
    formatted_options = "\n".join([f"{label_map[i]}) {opt}" for i, opt in enumerate(clean_opts)])

    # Construct the chat messages
    messages = [
        {"role": "system", "content": "You are a legal expert specializing in US case law."},
        {"role": "user", "content": f"CONTEXT: {clean_ctx}\n\nOPTIONS:\n{formatted_options}\n\nWhich option is the correct holding?"}
    ]

    # Apply template and tokenize
    # add_generation_prompt=True adds the header for the assistant to start speaking
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    return inputs, label_map[example["label"]]


# CHAT / INFERENCE FUNCTION
def run_legal_chat(model, tokenizer, dataset, num_samples=1):
    """
    Picks random samples from the test set and generates a response.
    """
    for _ in range(num_samples):
        # Resolve device
        try:
            device = next(model.parameters()).device
        except StopIteration:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        # Pick a random example
        random_idx = random.randint(0, len(dataset) - 1)
        example = dataset[random_idx]

        print(f"\n{'='*60}")
        print(f"ANALYZING CASE ID: {random_idx}")
        print(f"CONTEXT PREVIEW: {example['context'][:200]}...")
        print("-" * 30)
        # Print the options
        for i, opt in enumerate(example["endings"]):
            label = ["A", "B", "C", "D", "E"][i]
            print(f"   {label}) {opt}")
        print(f"{'='*60}")

        # Prepare input
        inputs, ground_truth = format_prompt(example, tokenizer, device)

        # Generate response
        # Using greedy decoding (temperature=0.0) for precise answers
        outputs = model.generate(
            inputs,
            max_new_tokens=64,
            temperature=0.0,
            do_sample=False, # Set to False for pure deterministic greedy, True for low temp sampling
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated part only (skip input prompt)
        generated_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)

        print(f"\n MODEL PREDICTION:\n{generated_text.strip()}")
        print(f"\n GROUND TRUTH: {ground_truth}")

        if ground_truth in generated_text:
            print("\nRESULT: [CORRECT]")
        else:
            print("\nRESULT: [INCORRECT]")

In [ ]:
# Run the chat loop (change num_samples to test more cases)
run_legal_chat(model, tokenizer, eval_dataset_ch, num_samples=5)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



ANALYZING CASE ID: 327
CONTEXT PREVIEW: power to regulate the channels and instrumentalities of commerce includes the power to prohibit their use for harmful purposes, even if the targeted harm itself occurs outside the flow of commerce and...
------------------------------
   A) holding it is within congresss authority to regulate all intrastate possession of child pornography not just that which has traveled in interstate commerce or has been produced using materials that have traveled in interstate commerce applying gonzales v raich 545 us 1 125 sct 2195 162 led2d 1 2005
   B) recognizing that congress had the power to regulate boxcars that traveled exclusively intrastate because of their inherent mobility and connection to interstate commerce it is no objection to such an exertion of commerce clause power that the dangers intended to be avoided arise in whole or in part out of matters connected with intrastate commerce
   C) holding that statute which prohibited gun possession nea

##

## Expert 2 Fine-Tuning (LegalBench)

While CaseHOLD provides the model with vertical depth in judicial precedents, **LegalBench** serves as the foundation for horizontal breadth.

It is a comprehensive, collaboratively built benchmark designed to evaluate legal reasoning across a wide spectrum of tasks, ranging from statutory interpretation and contractual clause classification to rhetorical analysis. By training our second expert on LegalBench, we aim to instill generalized legal logic and robust instruction-following capabilities. This ensures that the final merged model is not merely a citation engine, but a versatile legal assistant capable of navigating diverse and complex juridical instructions.

**Cleanup:** release training/inference objects and clear GPU memory to avoid leaks and reduce VRAM usage.
Then, depending on the `RUN_TRAINING_legalBench` flag, either initialize the base model with a LoRA adapter for training, or load the previously fine-tuned Expert 2 model and enable optimized inference mode.



In [ ]:
import gc #garbage collector

# Cleanup
try:
  del model
except:
  pass
try:
    del trainer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

In [ ]:
if RUN_TRAINING_legalBench:
  # Load the base model and tokenizer
  model, tokenizer = upload_model(model_base)
  model = add_lora(model)
  print("Environment successfully initialized.")


else:
  print(f"Loading adapter from: {path_expert_2}")

  # Load base model, then attach the LoRA adapter
  model, tokenizer = load_base_with_lora_adapter(path_expert_2)

Loading adapter from: /content/drive/Shareddrives/MIRCV Project/llm2/expert2_legalBench
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


### Dataset Ingestion and Stratified Disjoint Splitting



This cell handles the ingestion of the `Equall/legalbench_instruct` dataset that is provided as a single monolithic block without pre-defined validation or test sets.







1.  **Addressing Data Leakage Risks**:

    * **The Problem:** LegalBench comprises distinct tasks type (e.g., issue, interpretation). A naive random shuffle could distribute examples from the same narrow sub-task across both training and testing sets, leading to inflated performance metrics where the model "memorizes" the task template rather than learning the underlying legal reasoning.
    * **The Solution:** The `balanced_disjoint_splits_by_task` function enforces **disjoint** subsets. It manages a mutable pool of indices (`task2idx`), ensuring that once a specific example is allocated to the Test set, it is physically removed from the pool available for Validation and Training.

2.  **Class Imbalance Mitigation**:
    The allocation logic (`take_balanced`) actively fights the natural skew of the dataset.
    * Instead of allowing the largest tasks to dominate the training data (which would bias the model towards those specific problem types), the algorithm calculates a `base` quota to ensure an approximately equal representation of every available legal task.
    
3.  **Data Hygiene**:
    Prior to splitting, a filtering step ensures that all entries contain valid strings for `inputs` and `answer`. This prevents training failures downstream caused by `None` types or empty strings, which are not uncommon in large-scale legal corpora.

In [ ]:
# LOAD LegalBench

dataset_lb = load_dataset("Equall/legalbench_instruct", split="train")

dataset_lb = dataset_lb.filter(
    lambda x: isinstance(x.get("inputs"), str) and isinstance(x.get("answer"), str)
              and len(x["inputs"].strip()) > 0 and len(x["answer"].strip()) > 0
)

dataset_lb = dataset_lb.shuffle(seed=42)

README.md:   0%|          | 0.00/425 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/34.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90394 [00:00<?, ? examples/s]

Filter:   0%|          | 0/90394 [00:00<?, ? examples/s]

In [ ]:
def balanced_disjoint_splits_by_task(
    dataset: Dataset,
    task_col: str = "task_type",
    train_size: int = 8000,
    val_size: int = 500,
    test_size: int = 1000,
    seed: int = 42,
):
    # Group indices by task_type
    task2idx = defaultdict(list)
    for i, t in enumerate(dataset[task_col]):
        task2idx[t].append(i)

        #every type must have enough examples to serve all splits
    total = train_size + val_size + test_size
    n = len(task2idx)

    per_type = total // n
    rem = total % n

    # some types may need one extra example if total not divisible by n
    max_needed = per_type + (1 if rem > 0 else 0)

    for typ, idxs in task2idx.items():
        assert len(idxs) >= max_needed, (
            f"{typ} has only {len(idxs)} examples; need at least {max_needed} "
            f"to create disjoint balanced splits of total={total} across n={n} types"
        )


    rng = np.random.default_rng(seed)

    # Shuffle indices within each type
    for t in task2idx:
        rng.shuffle(task2idx[t])

    types = list(task2idx.keys())
    n = len(types) #5

    def take_balanced(pool, target_size):
        per_type = target_size // n
        rem = target_size % n

        rng.shuffle(types)  # randomize who gets the +1
        take_counts = {t: per_type for t in types}
        for t in types[:rem]:
            take_counts[t] += 1

        selected = []
        for t in types:
            k = take_counts[t]
            selected.extend(pool[t][:k])
            del pool[t][:k]
        rng.shuffle(selected)
        return selected

    # Mutable pool to keep splits disjoint
    pool = {t: idxs.copy() for t, idxs in task2idx.items()}

    test_idx  = take_balanced(pool, test_size)
    val_idx   = take_balanced(pool, val_size)
    train_idx = take_balanced(pool, train_size)

    return dataset.select(train_idx), dataset.select(val_idx), dataset.select(test_idx)




In [ ]:

train_dataset_lb, eval_dataset_lb, test_dataset_lb = balanced_disjoint_splits_by_task(
    dataset_lb,
    task_col="task_type",
    train_size=8000,
    val_size=500,
    test_size=1000,
    seed=42,
)

print("Balanced splits created.")
print("Train size:", len(train_dataset_lb))
print("Val size  :", len(eval_dataset_lb))
print("Test size :", len(test_dataset_lb))
print("Columns   :", train_dataset_lb.column_names)

# Quick sanity check: distribution (top 10 tasks)
from collections import Counter

print("FULL:", Counter(dataset_lb["task_type"]))
print("TRAIN:", Counter(train_dataset_lb["task_type"]))
print("VAL:", Counter(eval_dataset_lb["task_type"]))
print("TEST:", Counter(test_dataset_lb["task_type"]))

Balanced splits created.
Train size: 8000
Val size  : 500
Test size : 1000
Columns   : ['answer', 'index', 'task_type', 'task_name', 'inputs']
FULL: Counter({'interpretation': 60520, 'issue': 11599, 'rule': 9809, 'rhetoric': 6239, 'conclusion': 2227})
TRAIN: Counter({'rule': 1600, 'conclusion': 1600, 'issue': 1600, 'interpretation': 1600, 'rhetoric': 1600})
VAL: Counter({'rhetoric': 100, 'conclusion': 100, 'issue': 100, 'rule': 100, 'interpretation': 100})
TEST: Counter({'interpretation': 200, 'rule': 200, 'issue': 200, 'conclusion': 200, 'rhetoric': 200})


In [ ]:
# FORMAT LegalBench INTO LLAMA CHAT TEMPLATE
def formatting_prompts_func_legalbench(examples):
      texts = []
      for inp, ans in zip(examples["inputs"], examples["answer"]):
          inp = clean_legal_text(inp)
          ans = clean_legal_text(ans)

          convo = [
              {"role": "system", "content": "You are a helpful legal assistant. Follow the user's instructions and answer in the required format."},
              {"role": "user", "content": inp},
              {"role": "assistant", "content": ans},
          ]

          texts.append(tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False))

      return {"text": texts}

train_dataset_lb = train_dataset_lb.map(
      formatting_prompts_func_legalbench,
      batched=True,
      remove_columns=train_dataset_lb.column_names,
      num_proc=2,
  )

eval_dataset_raw_lb = eval_dataset_lb

eval_dataset_lb = eval_dataset_lb.map(
      formatting_prompts_func_legalbench,
      batched=True,
      remove_columns=eval_dataset_lb.column_names,
      num_proc=2,
  )

# for the test db we still need the format input/answer

print("Datasets formatted.")
print(train_dataset_lb[0]["text"][:500])
print(eval_dataset_lb[0]["text"][:500])



Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

Datasets formatted.
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful legal assistant. Follow the user's instructions and answer in the required format.<|eot_id|><|start_header_id|>user<|end_header_id|>

Answer the following questions considering the state of international law on January 1st, 2020. Question: Consider the country of Palau. Does the country provide for facilitated acquisition of citizenship (on easier terms th
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a helpful legal assistant. Follow the user's instructions and answer in the required format.<|eot_id|><|start_header_id|>user<|end_header_id|>

Label "Yes" if the paragraph interprets a statue using a dictionary definition for terms in the statute. The paragraph must reference that it is using a dictionary, or discuss the logic behind using a di

### Training

In [ ]:
checkpoint_ex2_path = BASE_PATH + "checkpoint_ex2"
log_file_path_lb = BASE_PATH + "training_logs_legalBench.jsonl"

The configuration of the fine-tuning training **mirrors Expert 1**: by adhering to the same training strategy, optimizer setup, and adapter structure as Expert 1, Expert 2 produces a LoRA adapter that is fully compatible for subsequent merging, enabling the construction of a unified model that combines case-law expertise with general legal reasoning capabilities.


In [ ]:
# TRAINING Expert 2
if RUN_TRAINING_legalBench:

  run_name = "expert-2-legalbench-instruct-lora"
  wandb.init(project="Legal-Model-Merging", name=run_name, mode="disabled")

  trainer2 = SFTTrainer( #supervised fine tuning trainer
      model=model,
      tokenizer=tokenizer,
      train_dataset=train_dataset_lb,
      eval_dataset=eval_dataset_lb,
      dataset_text_field="text",
      max_seq_length=max_seq_length,
      data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
      dataset_num_proc=2,
      callbacks=[
        EarlyStoppingCallback(early_stopping_patience=2),
        DriveLogCallback(log_file_path_lb)
      ],
      args=TrainingArguments(
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          gradient_accumulation_steps=16,
          num_train_epochs=1,

          learning_rate=2e-5,
          warmup_ratio=0.03,
          weight_decay=0.01,

          eval_strategy="steps",
          eval_steps=250,
          save_strategy="steps",
          save_steps=250,
          save_total_limit=1,
          load_best_model_at_end=True,
          metric_for_best_model="eval_loss",
          greater_is_better=False,

          logging_steps=10,
          report_to="none",

          fp16=not torch.cuda.is_bf16_supported(),
          bf16=torch.cuda.is_bf16_supported(),
          optim="adamw_8bit",
          lr_scheduler_type="cosine",
          output_dir=checkpoint_ex2_path,
      ),
  )




In [ ]:
if RUN_TRAINING_legalBench:
  trainer2_stats = trainer2.train()
  print("Expert 2 training completed.")

else:
  print_training_logs(log_file_path_lb)

Step       | Training Loss   | Validation Loss
---------------------------------------------
250        | 1.097000        | 0.911599       
500        | 1.212000        | 0.863097       


In [ ]:
#  SAVE Expert 2 ADAPTER
if RUN_TRAINING_legalBench:
  model.save_pretrained(path_expert_2)
  tokenizer.save_pretrained(path_expert_2)

  print(f"Expert 2 saved.")


### Evaluation Strategy and Metric Selection

The evaluation of Expert 2 is designed to be **task-agnostic**, reflecting the heterogeneous nature of LegalBench, which combines binary decisions, multiple-choice questions, ordinal labels, and free-form legal reasoning.
Rather than relying on a single metric, the evaluation framework uses a complementary **set of metrics**, each targeting a different failure mode of large language models in legal tasks.

1. **`Strict Exact Match (Strict EM)`**:

    Measures whether the normalized model prediction exactly matches the normalized ground-truth answer. This metric captures perfect correctness, which is crucial for legal tasks where even small deviations (extra words, reformulations) may change the legal meaning or violate task instructions but penalizes semantically correct but verbose answers.

2. **`Contains Ground Truth (Contains)`**:

    Checks whether the normalized ground-truth answer appears as a substring of the normalized prediction. This metric compensates for the rigidity of Strict EM but can overestimate correctness in cases where the ground truth appears coincidentally or without being the actual answer.

3. **`Label Accuracy (Label Acc)`**:

    Computed *only* for examples whose ground truth is label-like (binary yes/no, single-letter options, or ordinal numbers). This metric evaluates the model’s ability to *comply with instructions*.

4. **`ROUGE-L F1 (Non-Label Tasks Only)`**:

    For non-label, free-form answers, ROUGE-L F1 is computed based on the *Longest Common Subsequence (LCS)* between prediction and ground truth. This Metric captures structural and semantic overlap while tolerating paraphrasing, making it suitable for legal explanations and reasoning-heavy tasks.

Together, these metrics provide a balanced and realistic assessment of Expert 2’s capabilities across diverse legal tasks.

In [ ]:
from typing import Callable, Optional, Dict, Any


# Text + scoring helpers

def normalize_text(s: str) -> str:
    if s is None:
        return ""
    s = s.strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = s.strip(" \t\n\r\f\v")
    return s

def extract_binary_label(pred: str):
    """
    Extract 'yes'/'no' ONLY if it is the FIRST WORD of the prediction.
    Examples matched:
      - "Yes"
      - "Yes."
      - "No, because ..."
      - "no - ..."
    Examples NOT matched:
      - "There is no evidence..."
      - "Answer: yes"
      - "I think the answer is no"
    """
    p = normalize_text(pred)
    m = re.match(r"^(yes|no)\b", p)   # match at start only
    if m:
        return m.group(1)
    return None


def extract_single_letter(pred: str, allowed=("a","b","c","d","e","f","g","h","i","j")):
    """
    Extract a single option letter ONLY if it is at the beginning of the prediction
    and appears as a standalone token (not inside a word).

    Matches:
      - "A"
      - "b."
      - "C)"
      - "d - because ..."
      - "e: ..."

    Does NOT match:
      - "because ..."      (letter inside word)
      - "abc"              (not standalone)
    """
    p = normalize_text(pred)

    # first token must be a standalone letter in allowed, optionally followed by punctuation
    m = re.match(r"^\s*([a-j])\b", p)
    if m:
        letter = m.group(1)
        return letter if letter in allowed else None

    return None

def extract_single_number_0_4(pred: str):
    """
    Extract a numeric label in {0,1,2,3,4} ONLY if it is the FIRST token
    and appears as a standalone number (not inside another number/word).

    Matches:
      - "0"
      - "3."
      - "4) because ..."
      - "2 - ..."

    Does NOT match:
      - "10"             (not in 0-4)
      - "3rd"            (not standalone)
    """
    p = normalize_text(pred)
    m = re.match(r"^\s*([0-4])\b", p)
    if m:
        return m.group(1)
    return None


# ROUGE-L Longest Common Subsequence (LCS) helpers

def lcs_length(a_tokens, b_tokens):
    """
    Compute LCS length between two token lists using DP.
    O(len(a)*len(b)) time, O(min(len(a), len(b))) memory.
    """
    # use the shorter one to reduce memory
    if len(a_tokens) < len(b_tokens):
        short, long = a_tokens, b_tokens
    else:
        short, long = b_tokens, a_tokens

    prev = [0] * (len(short) + 1)
    for tok in long:
        curr = [0] * (len(short) + 1)
        for j in range(1, len(short) + 1):
            if tok == short[j - 1]:
                curr[j] = prev[j - 1] + 1
            else:
                curr[j] = max(prev[j], curr[j - 1])
        prev = curr
    return prev[-1]

def rouge_l_f1(pred: str, gt: str, max_tokens: int = 200) -> float:
    """
    Token-based ROUGE-L F1 between prediction and ground truth.
    Truncates to max_tokens for speed.
    """
    pred_toks = normalize_text(pred).split()[:max_tokens]
    gt_toks   = normalize_text(gt).split()[:max_tokens]

    if len(pred_toks) == 0 and len(gt_toks) == 0:
        return 1.0
    if len(pred_toks) == 0 or len(gt_toks) == 0:
        return 0.0

    L = lcs_length(pred_toks, gt_toks)
    P = L / len(pred_toks)
    R = L / len(gt_toks)
    return 0.0 if (P + R) == 0 else (2 * P * R / (P + R))



def score_prediction(pred: str, gt: str) -> dict:
    """
    Metrics:
    - strict_em: normalized pred == normalized gt
    - contains: normalized gt contained in normalized pred
    - label_acc: computed only if GT is label-like (binary yes/no, letter a-j, or number 0-4).
      For label_acc we extract label from both GT and PRED using STRICT rules:
      the label must be the FIRST token.
    - rougeL_f1: computed only if GT is non-label-like.
    """
    pred_n = normalize_text(pred)
    gt_n   = normalize_text(gt)

    out = {
        "strict_em": int(pred_n == gt_n),
        "contains": int(gt_n != "" and gt_n in pred_n),
        "label_acc": None,
    }

    # Determine label type from GT (STRICT)
    gt_bin = extract_binary_label(gt_n)
    if gt_bin is not None:
        pred_bin = extract_binary_label(pred_n)
        out["label_acc"] = int(pred_bin == gt_bin) if pred_bin is not None else 0
        return out

    gt_letter = extract_single_letter(gt_n)  # uses allowed a-j by default
    if gt_letter is not None:
        pred_letter = extract_single_letter(pred_n)
        out["label_acc"] = int(pred_letter == gt_letter) if pred_letter is not None else 0
        return out

    gt_num = extract_single_number_0_4(gt_n)
    if gt_num is not None:
        pred_num = extract_single_number_0_4(pred_n)
        out["label_acc"] = int(pred_num == gt_num) if pred_num is not None else 0
        return out

    # Not a label-like example
    out["label_acc"] = None
    return out



def pct(num, den):
    return 0.0 if den == 0 else (100.0 * num / den)



# Main evaluation function

def evaluate_general_capabilities(
    model,
    tokenizer,
    eval_dataset,
    using_test_set: bool = True,
    clean_fn: Optional[Callable[[str], str]] = None,
    system_prompt: str = "You are a helpful legal assistant. Follow the user's instructions and answer in the required format.",
    device: Optional[str] = None,
    num_samples: Optional[int] = 500,
    max_new_tokens: int = 128,
    gen_kwargs: Optional[Dict[str, Any]] = None,
    show_examples: int = 0,
    compute_rougeL: bool = True,
    rougeL_max_tokens: int = 200,
) -> Dict[str, Any]:
    """
    Task-agnostic evaluation on a RAW LegalBench-Instruct evaluation split.
    Expects eval_dataset items to have at least: 'inputs' and 'answer'.

    Metrics:
      - Strict EM (overall)
      - Contains (overall)
      - Label Accuracy (only on label-type examples)
      - ROUGE-L F1 (only on non-label examples, optional)

    Returns a dict with metrics and optional examples.
    """

    if clean_fn is None:
        clean_fn = lambda x: x  # no-op

    try:
        device = next(model.parameters()).device
    except StopIteration:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    if gen_kwargs is None:
        gen_kwargs = {}

    model.eval()

    split_name = "Test" if using_test_set else "Evaluation"
    print(f"\n--- Starting Expert 2 Evaluation on LegalBench-Instruct ({split_name.lower()} split) ---")
    print(f"Using existing {split_name} split -> size: {len(eval_dataset)}")
    eval_raw = eval_dataset
    if num_samples is not None:
        eval_raw = eval_raw.select(range(min(num_samples, len(eval_raw))))
        print(f"Evaluating on a subset: {len(eval_raw)} samples")

    overall = defaultdict(int)
    examples_out = []

    # for ROUGE-L aggregation (float)
    rougeL_sum = 0.0
    rougeL_n = 0

    # default generate args + user overrides
    generate_args = dict(
        max_new_tokens=max_new_tokens,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    generate_args.update(gen_kwargs)

    # takes raw dataset to save ground truth
    for ex in tqdm(eval_raw):
        user_inp = clean_fn(ex["inputs"])
        gt_ans   = clean_fn(ex["answer"])

        convo = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_inp},
        ]

        input_text = tokenizer.apply_chat_template(
            convo, tokenize=False, add_generation_prompt=True
        )
        inputs = tokenizer([input_text], return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(**inputs, **generate_args)

        pred_text = tokenizer.batch_decode(
            outputs[:, inputs.input_ids.shape[1]:],
            skip_special_tokens=True
        )[0]

        scores = score_prediction(pred_text, gt_ans)

        # aggregate overall metrics
        overall["n"] += 1
        overall["strict_em"] += scores["strict_em"]
        overall["contains"] += scores["contains"]

        if scores["label_acc"] is not None:
            overall["label_n"] += 1
            overall["label_correct"] += scores["label_acc"]
        else:
            # non-label example -> compute ROUGE-L F1
            if compute_rougeL:
                rougeL_sum += rouge_l_f1(pred_text, gt_ans, max_tokens=rougeL_max_tokens)
                rougeL_n += 1

        if show_examples and len(examples_out) < show_examples:
            examples_out.append({
                "prompt": user_inp,
                "gt": gt_ans,
                "pred": pred_text
            })

    # final metrics
    strict_em_pct = pct(overall["strict_em"], overall["n"])
    contains_pct  = pct(overall["contains"], overall["n"])
    label_acc_pct = pct(overall["label_correct"], overall["label_n"])

    rougeL_pct = None
    if compute_rougeL:
        rougeL_pct = 0.0 if rougeL_n == 0 else (100.0 * rougeL_sum / rougeL_n)

    # Print results (ONLY overall)
    print("\n=== Overall metrics (Expert 2 / LegalBench-Instruct eval) ===")
    print(f"Samples              : {overall['n']}")
    print(f"Strict EM (%)        : {strict_em_pct:.2f}")
    print(f"Contains GT (%)      : {contains_pct:.2f}")
    print(f"Label Accuracy (%)   : {label_acc_pct:.2f}  (on {overall['label_n']} label-type samples)")
    if compute_rougeL:
        print(f"ROUGE-L F1 (%)       : {rougeL_pct:.2f}  (on {rougeL_n} non-label samples)")
    print("\n--- Evaluation complete. ---")

    if show_examples:
        print("\n=== Qualitative sanity-check ===")
        for i, ex in enumerate(examples_out, 1):
            p = ex["prompt"]
            print(f"\n--- Example {i} ---")
            print("PROMPT:\n", p[:800], "..." if len(p) > 800 else "")
            print("GT:\n", ex["gt"])
            print("PRED:\n", ex["pred"])

    out = {
        "n": int(overall["n"]),
        "strict_em": float(strict_em_pct),
        "contains": float(contains_pct),
        "label_acc": float(label_acc_pct),
        "label_n": int(overall["label_n"]),
        "examples": examples_out,
    }

    if compute_rougeL:
        out.update({
            "rougeL_f1": float(rougeL_pct),
            "rougeL_n": int(rougeL_n),
        })

    return out


In [ ]:
RESULTS_FILE_lb = BASE_PATH + "evaluation_results_expert2.json"

# Flag to decide whether to perform evaluation or load existing results
PERFORM_EVALUATION_lb = not os.path.exists(RESULTS_FILE_lb)

In [ ]:
# Deterministic eval settings (fair comparison)
GEN_KWARGS_EVAL = {"do_sample": False, "temperature": 0.0}

def eval_model2(model, tokenizer, eval_dataset, use_adapter: bool, using_test_set: bool = True):
    """
    Wrapper to match your existing evaluation call signature.
    Returns the metrics dict produced by evaluate_general_capabilities.
    """
    if use_adapter:
        return evaluate_general_capabilities(
            model=model,
            tokenizer=tokenizer,
            eval_dataset=eval_dataset,
            using_test_set=using_test_set,
            clean_fn=clean_legal_text,
            num_samples=len(eval_dataset),
            max_new_tokens=128,
            gen_kwargs=GEN_KWARGS_EVAL,
            show_examples=0
        )
    else:
        # Evaluate base behavior by disabling the LoRA adapter temporarily
        with model.disable_adapter():
            return evaluate_general_capabilities(
                model=model,
                tokenizer=tokenizer,
                eval_dataset=eval_dataset,
                using_test_set=using_test_set,
                clean_fn=clean_legal_text,
                num_samples=len(eval_dataset),
                max_new_tokens=128,
                gen_kwargs=GEN_KWARGS_EVAL,
                show_examples=0
            )

In [ ]:
if PERFORM_EVALUATION_lb:
    # Evaluate Fine-Tuned Model
    acc_finetuned = eval_model2(model, tokenizer, eval_dataset_raw_lb, use_adapter=True, using_test_set=False)

    # Evaluate Base Model (Temporarily disabling LoRA adapters)
    acc_base = eval_model2(model, tokenizer, eval_dataset_raw_lb, use_adapter=False, using_test_set=False)

    print("\n=== Delta (FT - BASE) ===")
    print(f"Strict EM Δ (%)      : {acc_finetuned['strict_em'] - acc_base['strict_em']:+.2f}")
    print(f"Contains GT Δ (%)    : {acc_finetuned['contains'] - acc_base['contains']:+.2f}")
    print(f"Label Acc Δ (%)      : {acc_finetuned['label_acc'] - acc_base['label_acc']:+.2f} "
          f"(label_n={acc_finetuned['label_n']})")
    print(f"ROUGE-L Δ (%)        : {acc_finetuned['rougeL_f1'] - acc_base['rougeL_f1']:+.2f} ")

else:
    print(f"Skipping evaluation as results file already exists at: {RESULTS_FILE_lb}")
    # Initialize acc_finetuned and acc_base to None, as they won't be used for saving
    acc_finetuned = None
    acc_base = None

Skipping evaluation as results file already exists at: /content/drive/Shareddrives/MIRCV Project/llm2/evaluation_results_expert2.json


In [ ]:
def save_results_expert2(base_metrics, expert_metrics, eval_size, results_file):
    """
    Save Expert 2 (LegalBench-Instruct) results to JSON.
    base_metrics/expert_metrics are dicts from evaluate_general_capabilities.
    """
    data = {
        "base_model_metrics_legalbench": base_metrics,
        "expert2_model_metrics_legalbench": expert_metrics,
        "eval_samples_legalbench": eval_size
    }

    with open(results_file, "w") as f:
        json.dump(data, f, indent=4)

    print(f"Results successfully saved to: {results_file}")


def load_and_display_results_expert2(results_file, primary_metric="strict_em"):
    """
    Load results and display a comparative table for Expert 2.
    """
    if not os.path.exists(results_file):
        print(f"No results file found at {results_file}")
        return

    with open(results_file, "r") as f:
        data = json.load(f)

    base = data["base_model_metrics_legalbench"]
    expert = data["expert2_model_metrics_legalbench"]
    eval_size = data["eval_samples_legalbench"]


    imp_primary = expert[primary_metric] - base[primary_metric]
    imp_contains = expert["contains"] - base["contains"]
    imp_label = expert["label_acc"] - base["label_acc"]

    results_df = pd.DataFrame({
        "Model": ["Llama-3.1-8B (Base)", "Expert 2 (LoRA Fine-Tuned)"],
        "Dataset": ["LegalBench-Instruct", "LegalBench-Instruct"],
        "Examples Used": [eval_size, eval_size],

        "Strict EM": [f"{base['strict_em']:.2f}%", f"{expert['strict_em']:.2f}%"],
        "Δ Strict EM": ["-", f"{(expert['strict_em']-base['strict_em']):+.2f}%"],

        "Contains": [f"{base['contains']:.2f}%", f"{expert['contains']:.2f}%"],
        "Δ Contains": ["-", f"{imp_contains:+.2f}%"],

        "Label Acc": [f"{base['label_acc']:.2f}%", f"{expert['label_acc']:.2f}%"],
        "Δ Label Acc": ["-", f"{imp_label:+.2f}%"],

        "Label n": [base["label_n"], expert["label_n"]],

        "ROUGE-L": [f"{base['rougeL_f1']:.2f}%", f"{expert['rougeL_f1']:.2f}%"],
        "Δ ROUGE-L": ["-", f"{expert['rougeL_f1']-base['rougeL_f1']:+.2f}%"],
    })

    print("\n\n FINAL COMPARISON RESULTS (Expert 2):")
    display(results_df)


In [ ]:
if PERFORM_EVALUATION_lb:

    test_size = acc_finetuned["n"]

    # Save the results to Drive
    save_results_expert2(acc_base, acc_finetuned, test_size, RESULTS_FILE_lb)
    print("Results have been evaluated and saved. Displaying now:")
else:
    print("Results file already exists. Loading and displaying previous results:")

# Load and display the table (this will handle both cases)
load_and_display_results_expert2(RESULTS_FILE_lb)


Results file already exists. Loading and displaying previous results:


 FINAL COMPARISON RESULTS (Expert 2):


,Model,Dataset,Examples Used,Strict EM,Δ Strict EM,Contains,Δ Contains,Label Acc,Δ Label Acc,Label n,ROUGE-L,Δ ROUGE-L
0,Llama-3.1-8B (Base),LegalBench-Instruct,500,46.00%,-,63.20%,-,59.44%,-,429,47.20%,-
1,Expert 2 (LoRA Fine-Tuned),LegalBench-Instruct,500,67.20%,+21.20%,68.60%,+5.40%,70.16%,+10.72%,429,51.73%,+4.53%


The LoRA fine-tuning on LegalBench-Instruct leads to a clear and consistent performance improvement over the base Llama-3.1-8B model across all evaluated metrics. Overall, Expert 2 appears to convert many “partially correct” responses into exactly correct outputs, strengthening reliability on structured legal tasks. The results support the effectiveness of LoRA specialization, while also highlighting remaining limitations in robustness and free-form legal reasoning

## Adapter Merging (TIES / DARE / DARE+TIES)

Adapter merging is a technique that allows combining two or more models,
each trained to solve different tasks, into a single model capable of
performing all tasks **without requiring additional training**.

In this project, we have already fine-tuned two LoRA experts, each specialized
in a specific task. The goal of this section is to merge them into a single
expert that performs well on both tasks, avoiding the cost of retraining.

A key characteristic of this approach is that the merge is performed **directly
on the LoRA adapters**, rather than on the full base model weights.

This section implements multiple merging strategies (TIES, DARE, and DARE+TIES),
sweeps over their hyperparameters, and saves the resulting merged adapters
along with a manifest used for batch evaluation.


You can choose the merge strategy via `MERGE_MODE`:
- `"ties"`: TIES only (Trim -> Elect Sign -> Merge)
- `"dare"`: DARE only (Drop+Rescale on deltas, then weighted average)
- `"dare_ties"`: DARE preprocessing, then TIES

The output is a **single LoRA adapter** (`adapter_model.safetensors` + `adapter_config.json`).


In [ ]:
import os, json, math, copy
import torch
from safetensors.torch import load_file, save_file
from tqdm.auto import tqdm

In [ ]:
# ---- Reproducibility ----
set_seed(GLOBAL_SEED, deterministic=True)

In [ ]:
try:
  del model
except:
  pass
try:
    del trainer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

**Configuration**

Setting paths and hyperparameters.

Notes:
- `KEEP_RATIO` controls TIES TRIM (e.g., 0.1 keeps top 10% by |delta|).
- `DARE_DROP_RATE` is the drop probability p in DARE.
- `W1`, `W2` are the expert weights.
- `R_MERGE`, `ALPHA_OUT` define the rank/alpha of the output LoRA adapter.

In [ ]:
# ===== Paths (reuse llm_project variables) =====
# BASE_PATH is already defined above in the notebook
EXPERT1_PATH = model_casehold_path
EXPERT2_PATH = path_expert_2
MERGED_BASE_PATH = os.path.join(BASE_PATH, "merged_model")

# ===== Merge strategy sweep =====
MERGE_MODES = ["ties", "dare", "dare_ties"]

# ===== TIES hyperparameters =====
KEEP_RATIOS = [0.3, 0.6]          # keep top fraction by |delta|
EPS = 1e-8

# ===== DARE hyperparameters =====
DARE_DROP_RATES = [0.15, 0.3]      # drop ratio p
DARE_RESCALE = True                    # rescale by 1/(1-p)

# ===== Expert weights =====
W1, W2 = 1.0, 1.0

# ===== Output LoRA shape =====
R_MERGE = 32
ALPHA_OUT = 64.0

# ===== Runtime =====
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
OUT_DTYPE = torch.float16

# Defaults for "unused but harmless" parameters in sweeps
DEFAULT_KEEP_RATIO = KEEP_RATIOS[0]
DEFAULT_DARE_DROP_RATE = DARE_DROP_RATES[0]

print("DEVICE:", DEVICE)
print("MERGE_MODES:", MERGE_MODES)

DEVICE: cuda
MERGE_MODES: ['ties', 'dare', 'dare_ties']


We load:
- `adapter_config.json`
- `adapter_model.safetensors`

Then we find all `(lora_A, lora_B)` pairs and reconstruct the dense update matrix:

$$ \Delta W = (\alpha / r) \cdot B A $$

In [ ]:
def load_adapter(adapter_path):
  adapter_conf = json.load(open(os.path.join(adapter_path,"adapter_config.json"), "r", encoding = "utf-8" ))
  adapter_st = load_file(os.path.join(adapter_path,"adapter_model.safetensors"),device = "cpu")
  return adapter_conf, adapter_st

def find_pairs(state_dict):
  '''
  The state dict is taken from the safetensor file, which is a dictionary of the form:
  tensor_name -> tensor
  in a LoRA adapter, two matrices are saved for each target layer
  lora_A.weight
  lora_B.weight

  With this function, we find the A/B pairs and create a dictionary of the type
  layer_prefix -> keyA, keyB, tensorA, tensorB
  '''
  pairs_dict = {}
  for key_A, tensor_A in state_dict.items(): # searches among all tensors in the state_dict
    if "lora_A" in key_A and key_A.endswith("weight"): # looks for rows related to matrix A weights
      key_B = key_A.replace("lora_A","lora_B",1) # creates the key for B starting from A by replacing only the first occurrence of lora_A with lora_B
      if key_B not in state_dict: # if B does not exist in the state_dict, moves to the next module
        continue
      tensor_B = state_dict[key_B] # retrieves the tensor associated with B's key
      key_prefix = key_A.split("lora_A")[0].rstrip(".") # to create the dictionary key, it takes the part of A's key before 'lora_A' and removes the trailing dot
      pairs_dict[key_prefix] = (key_A, key_B, tensor_A, tensor_B) # saves the pair in the dictionary
  return pairs_dict


def compute_delta(A, B, alpha, r):
  '''
  Reconstructs the weight delta starting from matrices A and B
  alpha scales the update, r is the LoRA rank
  delta_W = (alpha/r)BA
  '''
  scale_factor = float(alpha)/float(r)
  A_ = A.to(torch.float32)
  B_ = B.to(torch.float32)

  delta = (B_ @ A_) * scale_factor
  return delta

### DARE (Drop And REscale)

**DARE motivation**

When merging multiple LoRA adapters or parameter deltas, a naive summation $\Delta W = \sum_{i=1}^{N} \Delta W_i$ often leads to:
- excessive weight scaling
- destructive interference between task-specific adaptations
- degradation of generalization

DARE introduces **controlled stochasticity** in the merging process
to:
- approximate a sparse ensemble of adaptations
- reduce co-adaptation between deltas
- preserve the expected magnitude of the merged parameters


**DARE Formulation**

Given a parameter delta $\Delta W$, DARE applies an element-wise Bernoulli mask:

$$
M_{ij} \sim \text{Bernoulli}(p)
$$

dropping elements using the created mask:

$$
\Delta\widetilde{ W}_{ij} = (1 - M_{ij}) \odot \Delta W_{ij}
$$

and rescaling the remaining deltas:

$$
\Delta\widehat{ W}_{ij} = \frac{\Delta\widetilde{ W}_{ij}}{(1-p)}
$$

where:
- $p \in [0,1]$ is the **drop rate**
- the rescaling factor $ \frac{1}{1 - p} $ ensures:

$$
\mathbb{E}[\Delta \widehat{W}] = \mathbb{E}[\Delta W]
$$

**The expectation-preserving rescaling** makes DARE an unbiased stochastic
regularization method applied during model merging.

The function `dare_delta` directly implements the theoretical definition of DARE


The following model merging techniques after DARE weight selection can be:
- weighted average
$$
\Delta W_{merged} = \frac{\lambda_1 \cdot \Delta W_1 + \lambda_2 \cdot \Delta W_2}{\lambda_1 + \lambda_2}
$$
where $\lambda$ is the scaling term to determine the importance of
the models to be merge
- a follow up merging method (eg. *TIES*)


In [ ]:
def dare_delta(input_tensor: torch.Tensor, mask_rate: float, use_rescale: bool, generator: Optional[torch.Generator] = None):
    """
    mask the input with mask rate
    :param input_tensor: Tensor, input tensor
    :param mask_rate: float, mask rate
    :param use_rescale: boolean, whether to rescale the input by 1 / (1 - mask_rate)
    """
    assert 0.0 <= mask_rate <= 1.0, f"wrong range of mask_rate {mask_rate}, should be [0.0, 1.0]!"

    mask = torch.bernoulli(torch.full_like(input=input_tensor, fill_value=mask_rate), generator=generator).to(input_tensor.device)
    masked_input_tensor = input_tensor * (1 - mask)

    if use_rescale and mask_rate != 1.0:
        masked_input_tensor = torch.div(input=masked_input_tensor, other=1 - mask_rate)
    return masked_input_tensor

### TIES Merging (TrIm, Elect Sign and Merge)

TIES is a merging algorithm designed to reduce **interference** between parameter updates, which can cause performance drops when combining multiple fine-tuned models. It mitigates this issue through three main steps:

1) **TRIM**: starting from the task vectors, it keeps only the **top k%** most significant changes (by magnitude) and discards the rest. In our implementation, *k* is the **keep_ratio** parameter.  
2) **ELECT SIGN**: it resolves sign conflicts across updates, since different models may push the same parameter in opposite directions. It builds a single **sign vector** that represents the dominant direction of change for each parameter.  
3) **MERGE**: it keeps only the values whose sign agrees with the elected sign (discarding zeros) and aggregates the remaining contributions using a **weighted mean**.

In our project we do not merge full model checkpoints, but **LoRA adapters**. Therefore, we apply TIES to the **effective weight updates** induced by LoRA.

- For each LoRA target module we reconstruct the update:
$$
  \Delta W = (\alpha / r)\,(B A)$$
  
  where $A$ and $B$ are the LoRA matrices, $r$ is the LoRA rank, and $\alpha$ is the LoRA scaling factor.

- We apply **TRIM / ELECT SIGN / MERGE** directly to the reconstructed $\Delta W\$ **for each layer**.

- After merging, $\Delta W_{\text{merged}}$ is generally no longer low-rank. To produce a deployable LoRA adapter, we approximate it with a rank-$r_{\text{out}}$ factorization using an approximate SVD, obtaining new matrices $A_{\text{out}}, B_{\text{out}}$ such that:
  $$
  \Delta W_{\text{merged}} \approx (\alpha_{\text{out}}/r_{\text{out}})\,(B_{\text{out}} A_{\text{out}})
  $$

In [ ]:
from typing import MutableMapping, Optional
def trim_delta(delta, keep_ratio = 0.2):
  '''
  Only some deltas are kept while others are set to zero.
  '''
  if keep_ratio >=1:
    return delta
  flat_delta = delta.abs().flatten() # calculate the absolute values of the deltas and transform the tensor into a vector of dimension out*in
  n = flat_delta.numel()
  k = max(1, int(keep_ratio * n)) # calculate the number of weights to keep (at least 1)
  min_threshold = torch.topk(flat_delta, k, largest = True).values.min() # take the smallest of the k largest values in magnitude of the delta vector
  mask = (delta.abs() >= min_threshold) # create the mask to keep only values greater than the threshold and set the rest to 0 in delta
  return delta * mask


def ties_merging(delta_1, delta_2, keep_ratio = 0.2, w1 = 1.0, w2 = 1.0, eps = 1e-8):
  '''
  The actual merging of the weights is performed.
  For each layer entry, the following steps are performed:
  1) retaining the largest values in magnitude
  2) taking the true sign
  3) discarding values that do not agree with the sign
  4) weighted average of the remaining values
  '''
  # perform trimming for the deltas of both models
  trim_1 = trim_delta(delta_1, keep_ratio)
  trim_2 = trim_delta(delta_2, keep_ratio)

  # for each entry, calculate the sign based on the weighted sum of the untrimmed deltas of the two models
  sign_score = w1 * trim_1 + w2 * trim_2
  true_sign = torch.sign(sign_score)

  # create two masks to select only the deltas that have the same sign as true_sign
  mask_1 = (torch.sign(trim_1) == true_sign) & (true_sign != 0) & (trim_1 != 0)
  mask_2 = (torch.sign(trim_2) == true_sign) & (true_sign != 0) & (trim_2 != 0)

  # weighted sum of deltas that passed the sign filter
  numerator = w1 * (trim_1 * mask_1) + w2 * (trim_2 * mask_2)
  # weighted sum of deltas that are actually active
  denominator = w1 * mask_1.to(numerator.dtype) + w2 * mask_2.to(numerator.dtype)
  # perform merging by taking the weighted average of the contributions that agree with the sign
  merged = torch.where(denominator > 0 , numerator / (denominator + eps), torch.zeros_like(numerator)) # eps serves to provide greater stability if the denominator is very small

  return merged

**Re-factorization into LoRA form via Randomized SVD**

After merging multiple adapters (e.g., via TIES or DARE), the resulting weight
update $\Delta W \in \mathbb{R}^{d_{\text{out}}\times d_{\text{in}}}$ is generally
**dense and full-rank**.

However, LoRA constrains updates to a low-rank structure:

$$
\Delta W_{\text{LoRA}} = \frac{\alpha}{r}\, B A,
\qquad
B \in \mathbb{R}^{d_{\text{out}}\times r},\;
A \in \mathbb{R}^{r\times d_{\text{in}}}
$$

To preserve LoRA’s **memory and compute efficiency**, the merged update is
approximated by its **best rank-$r$** representation.

---

**Low-rank approximation objective**

Define the rescaled matrix:
$$
M = \frac{\Delta W}{\alpha / r}
$$

The goal is to solve:
$$
M \approx U_r \Sigma_r V_r^\top
$$

where $(U_r, \Sigma_r, V_r)$ are the top-$r$ singular components of $M$.

---

**Why randomized SVD**

Computing an exact SVD of $M$ is computationally expensive for large layers.
Instead, **randomized SVD** efficiently approximates the dominant singular
subspace by:
- projecting $M$ onto a low-dimensional random subspace
- optionally refining it via power iterations

This yields an accurate rank-$r$ approximation at a fraction of the cost.

---

**Conversion to LoRA factors**

The truncated SVD is converted into LoRA parameters by symmetrically splitting
the singular values:

$$
B = U_r \Sigma_r^{1/2}, \qquad
A = \Sigma_r^{1/2} V_r^\top
$$

which ensures:
$$
\Delta W \approx \frac{\alpha}{r}\, BA
$$

This step restores a **LoRA-compatible adapter** that faithfully approximates
the merged update while maintaining the original LoRA parameterization.

---

**Key takeaway**

This procedure allows dense, merged updates to be:
- compressed back into low-rank LoRA form
- reused with standard LoRA inference pipelines
- compared fairly across different merging strategies


In [ ]:
def randomized_svd(M, rank, n_iter = 2, generator: Optional[torch.Generator] = None):
  '''
  An approximate SVD of a large matrix M is computed,
  without performing the full SVD which would be too computationally expensive.
  After TIES merging, a delta W is obtained which is no longer low-rank.
  To achieve the same as LoRA, we need to perform SVD.
  '''
  out_dim, in_dim = M.shape
  # create a random matrix
  Omega = torch.randn(in_dim, rank, device = M.device, dtype = M.dtype, generator=generator)
  # random combinations of M's columns are obtained
  Y = M @ Omega         # M: out*in, Omega: in*rank, Y: out*rank

  Q, _ = torch.linalg.qr(Y, mode = "reduced")

  # M*M_T has the left singular vectors of M as eigenvectors and eigenvalues equal to sigma squared.
  # The more times the multiplication is performed, the more the larger components are amplified, making them increasingly large and dominant.
  for _ in range(n_iter):
    Z = M.T @ Q
    Q, _ = torch.linalg.qr(M @ Z, mode = "reduced")

  # a smaller matrix is created on which to perform svd
  B = Q.T @ M         # Q_T: rank*out, M: out*in, B: rank*in
  # exact svd of matrix B. B = Ub*diag(S)*V_T
  Ub, S, Vh = torch.linalg.svd(B, full_matrices = False) #Ub: rank*rank, S: rank*1, Vh: rank*in
  # M approximately Q*Ub*S*V_T so U = Q*Ub
  U = Q @ Ub #Q: out*rank, Ub: rank*rank, U: out*rank

  return U[:, :rank], S[:rank], Vh[:rank,  :]

def create_new_lora(deltaW, r_out, alpha_out, n_iter = 2, generator: Optional[torch.Generator] = None):
  '''
  Creates a new LoRA adapter from the weight deltas calculated by merging the two previous LoRA adapters.
  '''
  scale_factor_out = float(alpha_out) / float(r_out)

  # deltaW = scale_factor_out * (B @ A)
  M = deltaW / scale_factor_out
  M = M.to(torch.float32)

  U, S, Vh = randomized_svd(M, rank = r_out, n_iter = n_iter, generator = generator)

  # force weights not to be negative, as this could happen due to approximation
  S = torch.clamp(S, min = 0.0)

  sqrtS= torch.sqrt(S)
  # B*A = (U*sqrtS)(sqrtS*V_T) = U*S*V_T approximately M
  B = U * sqrtS.unsqueeze(0)
  A = sqrtS.unsqueeze(1) * Vh

  A = A.to(deltaW.dtype)
  B = B.to(deltaW.dtype)

  return A,B

**Utility function**

In [ ]:
def build_out_dir(
    merged_base_path: str,
    merge_mode: str,
    keep_ratio: float,
    dare_drop_rate: float,
    r_merge: int,
    alpha_out: float,
) -> str:
    """
    Build a deterministic output directory name for merged adapters.

    merge_mode: "ties" | "dare" | "dare_ties"
    """
    def _fmt(x: float) -> str:
        # Example: 0.1 -> "01", 16.0 -> "160"
        return str(x).replace(".", "")

    if merge_mode == "ties":
        return f"{merged_base_path}_ties_kr{_fmt(keep_ratio)}_r{r_merge}_a{_fmt(alpha_out)}"
    elif merge_mode == "dare":
        return f"{merged_base_path}_dare_p{_fmt(dare_drop_rate)}_r{r_merge}_a{_fmt(alpha_out)}"
    elif merge_mode == "dare_ties":
        return f"{merged_base_path}_dare_ties_kr{_fmt(keep_ratio)}_p{_fmt(dare_drop_rate)}_r{r_merge}_a{_fmt(alpha_out)}"
    else:
        raise RuntimeError(f"Unknown merge_mode: {merge_mode}")


### DARE + TIES

In the **DARE + TIES** merging strategy, the process is decomposed into two stages:

1. **DARE**
   - stochastic dropout applied to the original delta  
   - control of the overall magnitude of the merged update  

2. **TIES (Trim, Elect, Sign)**  
   - selection of dominant parameter components over the survivor weights
   - resolution of sign conflicts across deltas  
   - mitigation of destructive interference  

Formally:

$$
\Delta W_{\text{final}} =
\text{TIES}(\text{DARE}(\Delta W_i))
$$

This combination integrates:
- **stochastic regularization** (DARE)
- **deterministic structure selection** (TIES)


**MERGING PIPELINE**

1) Load both adapters and configs.
2) For each shared LoRA module, reconstruct dense deltas for Expert 1 and Expert 2.
3) Apply the selected `MERGE_MODE`:
   - `ties`: TIES(delta1, delta2)
   - `dare`: average(DARE(delta1), DARE(delta2))
   - `dare_ties`: TIES(DARE(delta1), DARE(delta2))
4) Re-factorize the merged dense delta into a new LoRA (A,B) with rank `R_MERGE` and alpha `ALPHA_OUT`.
5) Save the merged adapter.

In [ ]:
import os, json, copy
from typing import Dict, Tuple, Optional
import torch
from tqdm.auto import tqdm
from safetensors.torch import save_file


def merge_lora_adapters_dare_ties(
    expert1_path: str,
    expert2_path: str,
    merged_base_path: str,
    merge_mode: str = "dare_ties",      # "ties" | "dare" | "dare_ties"
    keep_ratio: float = 0.1,            # TIES TRIM density
    dare_drop_rate: float = 0.9,        # DARE p
    dare_rescale: bool = True,          # rescale by 1/(1-p)
    r_merge: int = 16,                  # output LoRA rank
    alpha_out: float = 16.0,            # output LoRA alpha
    w1: float = 0.5,                    # weight expert1
    w2: float = 0.5,                    # weight expert2
    eps: float = 1e-8,                  # TIES numeric stability
    seed: int = 42,
    device: str = "cuda",
    out_dtype: torch.dtype = torch.float16,
    svd_n_iter: int = 2,
    use_randomized_svd: bool = True,
    verbose: bool = True,
) -> str:
    """
    Merge two LoRA adapters by:
      - Reconstruct dense deltas from LoRA factors for each module
      - Optional DARE preprocessing on deltas
      - Merge with TIES or weighted average (DARE-only)
      - Re-factorize merged dense delta back into LoRA factors with desired rank/alpha
      - Save merged adapter (adapter_model.safetensors + adapter_config.json)

    Returns:
      out_dir: str (where the merged adapter was saved)

    Expected helper functions (already in your notebook):
      - load_adapter(path) -> (conf: dict, sd: StateDict)
      - find_lora(sd) -> dict[module] = (keyA, keyB, A, B)
      - compute_dense_delta(A, B, alpha, r, device) -> dense delta tensor
      - dare_delta(delta, drop_rate, rescale, generator) -> delta tensor
      - ties_merge_two(delta1, delta2, keep_ratio, w1, w2, eps) -> merged delta tensor
      - lora_factors_from_delta(delta, r_out, alpha_out, n_iter, use_randomized, generator) -> (A_out, B_out)
    """
    # ---- Load adapters ----
    conf1, sd1 = load_adapter(expert1_path)
    conf2, sd2 = load_adapter(expert2_path)

    # ---- Sanity checks ----
    if verbose:
        for k in ["base_model_name_or_path", "peft_type", "task_type"]:
            if conf1.get(k, None) != conf2.get(k, None):
                print(f"[WARNING] adapter configuration mismatch on {k}: {conf1.get(k)} vs {conf2.get(k)}")

    # ---- Read per-adapter r/alpha for delta reconstruction ----
    r1, alpha_1 = int(conf1["r"]), float(conf1["lora_alpha"])
    r2, alpha_2 = int(conf2["r"]), float(conf2["lora_alpha"])

    pairs1 = find_pairs(sd1)
    pairs2 = find_pairs(sd2)

    modules_to_process = sorted(set(pairs1.keys()) | set(pairs2.keys()))

    if verbose:
        print(f"Shared LoRA modules: {len(modules_to_process)}")


    out_dir = build_out_dir(
        merged_base_path=merged_base_path,
        merge_mode=merge_mode,
        keep_ratio=keep_ratio,
        dare_drop_rate=dare_drop_rate,
        r_merge=r_merge,
        alpha_out=alpha_out
    )

    os.makedirs(out_dir, exist_ok=True)
    if verbose:
        print("Output dir:", out_dir)

    # ---- RNG for DARE / randomized SVD ----
    gen = torch.Generator(device=device)
    gen.manual_seed(seed)

    output_sd= {}

    # ---- Merge loop (per module) ----
    for module in tqdm(modules_to_process, desc="Merging modules", disable=not verbose):

        # Reconstruct dense deltas
        if module in pairs1:
          key_A_out, key_B_out, A1, B1 = pairs1[module]
        else:
          key_A_out, key_B_out, A2, B2 = pairs2[module]

        if module in pairs1:
          _, _, A1, B1 = pairs1[module]
          delta1 = compute_delta(A1, B1, alpha_1, r1).to(device)
        else:
          _, _, A2, B2 = pairs2[module]
          delta1 = torch.zeros_like(compute_delta(A2, B2, alpha_2, r2)).to(device)

        if module in pairs2:
          _, _, A2, B2 = pairs2[module]
          delta2 = compute_delta(A2, B2, alpha_2, r2).to(device)
        else:
          _, _, A1, B1 = pairs1[module]
          delta2 = torch.zeros_like(compute_delta(A1, B1, alpha_1, r1)).to(device)


        # Apply merge mode
        if merge_mode == "ties":
            merged_delta = ties_merging(delta1, delta2, keep_ratio=keep_ratio, w1=w1, w2=w2, eps=eps)

        elif merge_mode == "dare":
            d1 = dare_delta(delta1, mask_rate=dare_drop_rate, use_rescale=dare_rescale, generator=gen)
            d2 = dare_delta(delta2, mask_rate=dare_drop_rate, use_rescale=dare_rescale, generator=gen)

            denom = (w1 + w2)
            if denom == 0:
                raise ValueError("w1+w2 must be > 0 for averaging")
            merged_delta = (w1 * d1 + w2 * d2) / denom

        elif merge_mode == "dare_ties":
            d1 = dare_delta(delta1, mask_rate=dare_drop_rate, use_rescale=dare_rescale, generator=gen)
            d2 = dare_delta(delta2, mask_rate=dare_drop_rate, use_rescale=dare_rescale, generator=gen)
            merged_delta = ties_merging(d1, d2, keep_ratio=keep_ratio, w1=w1, w2=w2, eps=eps)

        else:
            raise RuntimeError(f"Unknown merge_mode: {merge_mode}")

        # Convert merged dense delta back to LoRA factors
        A_out, B_out = create_new_lora(
            merged_delta,
            r_out=r_merge,
            alpha_out=alpha_out,
            n_iter=svd_n_iter,
            generator=gen
        )

        # Save tensors using Expert 1 key names
        output_sd[key_A_out] = A_out.to(dtype=out_dtype, device="cpu")
        output_sd[key_B_out] = B_out.to(dtype=out_dtype, device="cpu")

        # Free per-layer tensors
        del delta1, delta2, merged_delta, A_out, B_out
        if device == "cuda":
            torch.cuda.empty_cache()

    # ---- Build output config ----
    out_conf = copy.deepcopy(conf1)
    out_conf["r"] = int(r_merge)
    out_conf["lora_alpha"] = float(alpha_out)

    # ---- Save adapter ----
    save_file(output_sd, os.path.join(out_dir, "adapter_model.safetensors"))
    with open(os.path.join(out_dir, "adapter_config.json"), "w", encoding="utf-8") as f:
        json.dump(out_conf, f, indent=2)

    if verbose:
        print("[SUCCESS] Merge finished. Saved merged LoRA adapter to:", out_dir)

    return out_dir


All merged models files are saved in separate directories.

At the end of the generation of all merged model a **MANIFEST** file is created listing all the created model with their parameters respectively and the corresponding directory

In [ ]:
MERGED_MANIFEST = os.path.join(BASE_PATH, "merged_adapters_manifest.json")
PERFORM_MERGING = not os.path.exists(MERGED_MANIFEST)

if PERFORM_MERGING:
    print("Performing merging sweep...")
    results = []  # list of dicts with metadata + output dir

    for mode in MERGE_MODES:
        if mode == "ties":
            # sweep keep_ratio only
            for kr in KEEP_RATIOS:
                out_dir = merge_lora_adapters_dare_ties(
                    expert1_path=EXPERT1_PATH,
                    expert2_path=EXPERT2_PATH,
                    merged_base_path=MERGED_BASE_PATH,
                    merge_mode="ties",
                    keep_ratio=kr,
                    dare_drop_rate=DEFAULT_DARE_DROP_RATE,   # unused here, but harmless
                    dare_rescale=DARE_RESCALE,
                    r_merge=R_MERGE,
                    alpha_out=ALPHA_OUT,
                    w1=W1, w2=W2,
                    eps=EPS,
                    device=DEVICE,
                    out_dtype=OUT_DTYPE,
                    svd_n_iter=2,
                    use_randomized_svd=True,
                    verbose=True,
                )
                results.append({
                    "mode": mode,
                    "keep_ratio": kr,
                    "dare_drop_rate": None,
                    "out_dir": out_dir,
                })

        elif mode == "dare":
            # sweep dare_drop_rate only
            for p in DARE_DROP_RATES:
                out_dir = merge_lora_adapters_dare_ties(
                    expert1_path=EXPERT1_PATH,
                    expert2_path=EXPERT2_PATH,
                    merged_base_path=MERGED_BASE_PATH,
                    merge_mode="dare",
                    keep_ratio=DEFAULT_KEEP_RATIO,  # unused here, but harmless
                    dare_drop_rate=p,
                    dare_rescale=DARE_RESCALE,
                    r_merge=R_MERGE,
                    alpha_out=ALPHA_OUT,
                    w1=W1, w2=W2,
                    eps=EPS,
                    device=DEVICE,
                    out_dtype=OUT_DTYPE,
                    svd_n_iter=2,
                    use_randomized_svd=True,
                    verbose=True,
                )
                results.append({
                    "mode": mode,
                    "keep_ratio": None,
                    "dare_drop_rate": p,
                    "out_dir": out_dir,
                })

        elif mode == "dare_ties":
            # sweep both keep_ratio and dare_drop_rate
            for kr in KEEP_RATIOS:
                for p in DARE_DROP_RATES:
                    out_dir = merge_lora_adapters_dare_ties(
                        expert1_path=EXPERT1_PATH,
                        expert2_path=EXPERT2_PATH,
                        merged_base_path=MERGED_BASE_PATH,
                        merge_mode="dare_ties",
                        keep_ratio=kr,
                        dare_drop_rate=p,
                        dare_rescale=DARE_RESCALE,
                        r_merge=R_MERGE,
                        alpha_out=ALPHA_OUT,
                        w1=W1, w2=W2,
                        eps=EPS,
                        device=DEVICE,
                        out_dtype=OUT_DTYPE,
                        svd_n_iter=2,
                        use_randomized_svd=True,
                        verbose=True,
                    )
                    results.append({
                        "mode": mode,
                        "keep_ratio": kr,
                        "dare_drop_rate": p,
                        "out_dir": out_dir,
                    })

    print(f"\nDone. Generated {len(results)} merged adapters.")
    for r in results[:10]:
        print(r)


    # ---- Save manifest for batch evaluation ----
    with open(MERGED_MANIFEST, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2)
    print("Saved manifest to:", MERGED_MANIFEST)
else:
    print(f"Skipping merging as results file already exists at: {MERGED_MANIFEST}")


Performing merging sweep...
Shared LoRA modules: 224
Output dir: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr03_r32_a640


Merging modules:   0%|          | 0/224 [00:00<?, ?it/s]

[SUCCESS] Merge finished. Saved merged LoRA adapter to: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr03_r32_a640
Shared LoRA modules: 224
Output dir: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr06_r32_a640


Merging modules:   0%|          | 0/224 [00:00<?, ?it/s]

[SUCCESS] Merge finished. Saved merged LoRA adapter to: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr06_r32_a640
Shared LoRA modules: 224
Output dir: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_p015_r32_a640


...

[SUCCESS] Merge finished. Saved merged LoRA adapter to: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_ties_kr03_p03_r32_a640
Shared LoRA modules: 224
Output dir: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_ties_kr06_p015_r32_a640


Merging modules:   0%|          | 0/224 [00:00<?, ?it/s]

[SUCCESS] Merge finished. Saved merged LoRA adapter to: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_ties_kr06_p015_r32_a640
Shared LoRA modules: 224
Output dir: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_ties_kr06_p03_r32_a640


Merging modules:   0%|          | 0/224 [00:00<?, ?it/s]

[SUCCESS] Merge finished. Saved merged LoRA adapter to: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_ties_kr06_p03_r32_a640

Done. Generated 8 merged adapters.
{'mode': 'ties', 'keep_ratio': 0.3, 'dare_drop_rate': None, 'out_dir': '/content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr03_r32_a640'}
{'mode': 'ties', 'keep_ratio': 0.6, 'dare_drop_rate': None, 'out_dir': '/content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr06_r32_a640'}
{'mode': 'dare', 'keep_ratio': None, 'dare_drop_rate': 0.15, 'out_dir': '/content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_p015_r32_a640'}
{'mode': 'dare', 'keep_ratio': None, 'dare_drop_rate': 0.3, 'out_dir': '/content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_p03_r32_a640'}
{'mode': 'dare_ties', 'keep_ratio': 0.3, 'dare_drop_rate': 0.15, 'out_dir': '/content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_ties_kr03_p015_r32_a640'}
{'mode': 'dare_ties', 'keep_ratio': 0.3,

## Evaluation of Merged Adapters

This section evaluates **all** merged adapters produced in the merging sweep on both CaseHOLD and LegalBench, saving one JSON per merged adapter and showing a summary table. This allows us to apply a manual grid search over the hyperparameters used for the merged models creation.

In [ ]:
import glob, gc

# -----------------------------
# Load baseline (already computed) results
# -----------------------------
with open(RESULTS_FILE_ch, "r") as f:
    _res_ch = json.load(f)
BASE_ACC_CH = float(_res_ch["base_model_accuracy_caseHOLD"])
EXPERT1_ACC_CH = float(_res_ch["expert_model_accuracy_caseHOLD"])

with open(RESULTS_FILE_lb, "r") as f:
    _res_lb = json.load(f)
BASE_LB = _res_lb["base_model_metrics_legalbench"]
EXPERT2_LB = _res_lb["expert2_model_metrics_legalbench"]

# -----------------------------
# Locate merged adapters
# -----------------------------
MERGED_MANIFEST = os.path.join(BASE_PATH, "merged_adapters_manifest.json")
if os.path.exists(MERGED_MANIFEST):
    with open(MERGED_MANIFEST, "r", encoding="utf-8") as f:
        manifest = json.load(f)
    merged_items = manifest
else:
    # Fallback: scan Drive for merged adapters
    merged_dirs = sorted(glob.glob(os.path.join(BASE_PATH, "merged_model*")))
    merged_items = [{"out_dir": d, "mode": None, "keep_ratio": None, "dare_drop_rate": None} for d in merged_dirs]

def _is_adapter_dir(d: str) -> bool:
    return os.path.exists(os.path.join(d, "adapter_model.safetensors")) and os.path.exists(os.path.join(d, "adapter_config.json"))

merged_items = [it for it in merged_items if _is_adapter_dir(it["out_dir"])]
print(f"Found {len(merged_items)} merged adapters to evaluate.")


# -----------------------------
# Per-adapter evaluation cache
# -----------------------------
MERGED_EVAL_DIR = os.path.join(BASE_PATH, "merged_evaluations")
os.makedirs(MERGED_EVAL_DIR, exist_ok=True)

def adapter_id(adapter_dir: str) -> str:
    return os.path.basename(adapter_dir.rstrip("/"))

def eval_cache_path(output_dir: str, adapter_dir: str) -> str:
    return os.path.join(output_dir, f"{adapter_id(adapter_dir)}.json")

def free_gpu():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

def evaluate_merged_adapter(adapter_dir: str, output_dir:str, meta: dict, datasets, using_test_set: bool, base_acc_ch=None, base_acc_lb=None, exp_acc_ch=None, exp_acc_lb=None):
    out_file = eval_cache_path(output_dir, adapter_dir)
    if os.path.exists(out_file):
        with open(out_file, "r", encoding="utf-8") as f:
            return json.load(f)
    dataset_ch = copy.deepcopy(datasets["ch"])
    dataset_lb = copy.deepcopy(datasets["lb"])
    print("\n" + "="*80)
    print("Evaluating:", adapter_dir)
    print("Meta:", {k: meta.get(k) for k in ["mode","keep_ratio","dare_drop_rate"]})

    model, tokenizer = load_base_with_lora_adapter(adapter_dir)

    # ---- CaseHOLD ----
    acc_ch = evaluate_model(model, tokenizer, dataset_ch, use_adapter=True, using_test_set=using_test_set)

    # ---- LegalBench ----
    metrics_lb = eval_model2(model, tokenizer, dataset_lb, use_adapter=True, using_test_set=using_test_set)

    result = {
        "adapter_dir": adapter_dir,
        "meta": meta,
        "casehold": {
            "base_acc": base_acc_ch,
            "expert1_acc": exp_acc_ch,
            "merged_acc": acc_ch,
            "delta_vs_base": acc_ch - base_acc_ch,
            "delta_vs_expert1": acc_ch - exp_acc_ch,
            "n": int(len(dataset_ch)),
        },
        "legalbench": {
            "base": base_acc_lb,
            "expert2": exp_acc_lb,
            "merged": metrics_lb,
            "n": int(metrics_lb.get("n", len(dataset_lb))),
        },
    }

    with open(out_file, "w", encoding="utf-8") as f:
        json.dump(result, f, indent=2)

    # cleanup
    del model
    free_gpu()

    return result



Found 8 merged adapters to evaluate.


In [ ]:
# -----------------------------
# Summary table (as a function)
# -----------------------------
def build_merged_summary_table(
    all_results: list,
    *,
    dataset_lb_dim,
    merged_eval_dir: str,
    sort_by=("casehold_acc", "legalbench_rougeL_f1"),
    ascending=False,
) -> pd.DataFrame:
    """Create + display + persist a summary table for merged adapters.

    Rows: one per merged adapter.
    Columns: merge hyperparameters + CaseHOLD accuracy + LegalBench metrics.

    Notes:
    - Metrics not present are left as NaN.
    - Saves the sorted table to `<merged_eval_dir>/summary_table.csv`.
    """

    def pick_float(m: dict, k: str):
        v = (m or {}).get(k, None)
        return np.nan if v is None else float(v)

    rows = []
    for r in all_results:
        lbm = r["legalbench"]["merged"]

        rows.append({
            "adapter": adapter_id(r.get("adapter_dir") or r.get("meta", {}).get("out_dir") or ""),
            "mode": (r.get("meta", {}) or {}).get("mode"),
            "keep_ratio": (r.get("meta", {}) or {}).get("keep_ratio"),
            "dare_drop_rate": (r.get("meta", {}) or {}).get("dare_drop_rate"),
            "casehold_acc": (r.get("casehold", {}) or {}).get("merged_acc", np.nan),
            "casehold_delta_vs_base": (r.get("casehold", {}) or {}).get("delta_vs_base", np.nan),
            "legalbench_strict_em": pick_float(lbm, "strict_em"),
            "legalbench_contains": pick_float(lbm, "contains"),
            "legalbench_label_acc": pick_float(lbm, "label_acc"),
            "legalbench_rougeL_f1": pick_float(lbm, "rougeL_f1"),
            "legalbench_n": int((lbm or {}).get("n", dataset_lb_dim)),
        })

    df = pd.DataFrame(rows)

    # Sort by a combined objective (customize as needed)
    df_sorted = df.sort_values(by=list(sort_by), ascending=ascending)

    # Save a global summary as well
    summary_file = os.path.join(merged_eval_dir, "summary_table.csv")
    df_sorted.to_csv(summary_file, index=False)
    print("Saved summary to:", summary_file)

    return df_sorted

In [ ]:
# -----------------------------
# Run evaluation over all merged adapters
# -----------------------------
PERFORM_MERGED_EVAL = not os.path.exists(os.path.join(MERGED_EVAL_DIR, "summary_table.csv"))

if PERFORM_MERGED_EVAL:
    print("\nEvaluating all merged adapters...")
    all_results = []
    eval_datasets = {"ch":eval_dataset_ch, "lb":eval_dataset_raw_lb}
    for it in merged_items:
        meta = {k: it.get(k) for k in ["mode", "keep_ratio", "dare_drop_rate", "out_dir"]}
        res = evaluate_merged_adapter(it["out_dir"], MERGED_EVAL_DIR, meta, eval_datasets, False, BASE_ACC_CH, BASE_LB, EXPERT1_ACC_CH, EXPERT2_LB)
        all_results.append(res)

    df_sorted = build_merged_summary_table(
        all_results,
        dataset_lb_dim = len(eval_dataset_raw_lb),
        merged_eval_dir=MERGED_EVAL_DIR,
    )

else:
    print(f"Skipping evaluation as results file already exists at: {MERGED_EVAL_DIR}")
    df_sorted = pd.read_csv(os.path.join(MERGED_EVAL_DIR, "summary_table.csv"))

print(f"Found {len(merged_items)} merged adapters to evaluate.")
display(df_sorted)



Evaluating all merged adapters...

Evaluating: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr03_r32_a640
Meta: {'mode': 'ties', 'keep_ratio': 0.3, 'dare_drop_rate': None}
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]


Starting evaluation on Evaluation set...


...

  0%|          | 0/500 [00:00<?, ?it/s]


=== Overall metrics (Expert 2 / LegalBench-Instruct eval) ===
Samples              : 500
Strict EM (%)        : 59.00
Contains GT (%)      : 59.80
Label Accuracy (%)   : 61.77  (on 429 label-type samples)
ROUGE-L F1 (%)       : 45.82  (on 71 non-label samples)

--- Evaluation complete. ---


,adapter,mode,keep_ratio,dare_drop_rate,casehold_acc,casehold_delta_vs_base,legalbench_strict_em,legalbench_contains,legalbench_label_acc,legalbench_rougeL_f1,legalbench_n
3,merged_model_dare_p03_r32_a640,dare,NaN,0.30,69.8,13.8,57.8,60.2,61.072261,41.240292,500
2,merged_model_dare_p015_r32_a640,dare,NaN,0.15,69.0,13.0,58.0,60.6,61.072261,42.648743,500
1,merged_model_ties_kr06_r32_a640,ties,0.6,NaN,48.8,-7.2,62.4,63.6,65.268065,47.058506,500
0,merged_model_ties_kr03_r32_a640,ties,0.3,NaN,47.2,-8.8,60.0,61.2,62.237762,48.476787,500
6,merged_model_dare_ties_kr06_p015_r32_a640,dare_ties,0.6,0.15,41.0,-15.0,61.2,62.0,64.102564,46.430691,500
7,merged_model_dare_ties_kr06_p03_r32_a640,dare_ties,0.6,0.30,40.8,-15.2,59.0,59.8,61.771562,45.817478,500
5,merged_model_dare_ties_kr03_p03_r32_a640,dare_ties,0.3,0.30,40.4,-15.6,59.2,59.6,61.771562,46.284726,500
4,merged_model_dare_ties_kr03_p015_r32_a640,dare_ties,0.3,0.15,39.6,-16.4,61.0,61.6,63.869464,46.369327,500


Saved summary to: /content/drive/Shareddrives/MIRCV Project/llm2/merged_evaluations/summary_table.csv
Found 8 merged adapters to evaluate.


,adapter,mode,keep_ratio,dare_drop_rate,casehold_acc,casehold_delta_vs_base,legalbench_strict_em,legalbench_contains,legalbench_label_acc,legalbench_rougeL_f1,legalbench_n
3,merged_model_dare_p03_r32_a640,dare,NaN,0.30,69.8,13.8,57.8,60.2,61.072261,41.240292,500
2,merged_model_dare_p015_r32_a640,dare,NaN,0.15,69.0,13.0,58.0,60.6,61.072261,42.648743,500
1,merged_model_ties_kr06_r32_a640,ties,0.6,NaN,48.8,-7.2,62.4,63.6,65.268065,47.058506,500
0,merged_model_ties_kr03_r32_a640,ties,0.3,NaN,47.2,-8.8,60.0,61.2,62.237762,48.476787,500
6,merged_model_dare_ties_kr06_p015_r32_a640,dare_ties,0.6,0.15,41.0,-15.0,61.2,62.0,64.102564,46.430691,500
7,merged_model_dare_ties_kr06_p03_r32_a640,dare_ties,0.6,0.30,40.8,-15.2,59.0,59.8,61.771562,45.817478,500
5,merged_model_dare_ties_kr03_p03_r32_a640,dare_ties,0.3,0.30,40.4,-15.6,59.2,59.6,61.771562,46.284726,500
4,merged_model_dare_ties_kr03_p015_r32_a640,dare_ties,0.3,0.15,39.6,-16.4,61.0,61.6,63.869464,46.369327,500


## Final Evaluation on the Hold-Out Test Set

In this final section we evaluate the **base model**, the two **fine-tuned experts**, and the **merged adapters** (those who performed better in both evaluations) on the hold-out test split. We persist all outputs (JSON/CSV) to Drive for reproducibility.

In [ ]:
TEST_DIR = BASE_PATH + "test_results/"
os.makedirs(TEST_DIR, exist_ok=True)

TEST_RESULTS_FILE_ch=os.path.join(TEST_DIR,"test_results_ch.json")
PERFORM_TEST_EVALUATION_ch = not os.path.exists(TEST_RESULTS_FILE_ch)

TEST_RESULTS_FILE_lb=os.path.join(TEST_DIR,"test_results_lb.json")
PERFORM_TEST_EVALUATION_lb = not os.path.exists(TEST_RESULTS_FILE_lb)


**Evaluate** base model and expert 1 on test set

In [ ]:
if PERFORM_TEST_EVALUATION_ch:
    print("\nPerforming final test evaluation on CaseHOLD...")

    model, tokenizer = load_base_with_lora_adapter(model_casehold_path)
    acc_finetuned_ch = evaluate_model(model, tokenizer, test_dataset_ch, use_adapter=True, using_test_set=True)
    acc_base_ch = evaluate_model(model, tokenizer, test_dataset_ch, use_adapter=False, using_test_set=True)

    print(f"\tTest set results - Base model accuracy: {acc_base_ch:.2f}%\n\t Fine-tuned model accuracy: {acc_finetuned_ch:.2f}%")

    save_results(acc_base_ch,acc_finetuned_ch,len(test_dataset_ch), TEST_RESULTS_FILE_ch)

    try:
      del model
    except:
      pass
    try:
        del trainer
    except:
        pass
    gc.collect()
    torch.cuda.empty_cache()

else:
    print(f"Skipping evaluation as results file already exists at: {TEST_RESULTS_FILE_ch}")
    # Initialize acc_finetuned_ch and acc_base_ch to None, as they won't be used for saving
    acc_finetuned_ch = None
    acc_base_ch = None


Performing final test evaluation on CaseHOLD...
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Starting evaluation on Test set...


  0%|          | 0/1000 [00:00<?, ?it/s]


Starting evaluation on Test set...


  0%|          | 0/1000 [00:00<?, ?it/s]

	Test set results - Base model accuracy: 55.70%
	 Fine-tuned model accuracy: 73.90%
 Results successfully saved to: /content/drive/Shareddrives/MIRCV Project/llm2/test_results/test_results_ch.json


**Evaluate** base model and expert 2 on test set

In [ ]:
if PERFORM_TEST_EVALUATION_lb:
    print("\nPerforming final test evaluation on LegalBench...")
    model, tokenizer = load_base_with_lora_adapter(path_expert_2)

    acc_finetuned_lb = eval_model2(model, tokenizer, test_dataset_lb, use_adapter=True, using_test_set=True)
    acc_base_lb = eval_model2(model, tokenizer, test_dataset_lb, use_adapter=False, using_test_set=True)

    print("\n=== Delta (FT - BASE) ===")
    print(f"Strict EM Δ (%)      : {acc_finetuned_lb['strict_em'] - acc_base_lb['strict_em']:+.2f}")
    print(f"Contains GT Δ (%)    : {acc_finetuned_lb['contains'] - acc_base_lb['contains']:+.2f}")
    print(f"Label Acc Δ (%)      : {acc_finetuned_lb['label_acc'] - acc_base_lb['label_acc']:+.2f} "
          f"(label_n={acc_finetuned_lb['label_n']})")
    print(f"ROUGE-L Δ (%)        : {acc_finetuned_lb['rougeL_f1'] - acc_base_lb['rougeL_f1']:+.2f} ")

    save_results_expert2(acc_base_lb,acc_finetuned_lb,len(test_dataset_lb), TEST_RESULTS_FILE_lb)

    try:
      del model
    except:
      pass
    try:
        del trainer
    except:
        pass
    gc.collect()
    torch.cuda.empty_cache()
else:
    print(f"Skipping evaluation as results file already exists at: {TEST_RESULTS_FILE_lb}")
    # Initialize acc_finetuned_lb and acc_base_lb to None, as they won't be used for saving
    acc_finetuned_lb = None
    acc_base_lb = None


Performing final test evaluation on LegalBench...
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

--- Starting Expert 2 Evaluation on LegalBench-Instruct (test split) ---
Using existing Test split -> size: 1000
Evaluating on a subset: 1000 samples


  0%|          | 0/1000 [00:00<?, ?it/s]

Unsloth: Input IDs of shape torch.Size([1, 1060]) with length 1060 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1401]) with length 1401 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1247]) with length 1247 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1392]) with length 1392 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 1037]) with length 1037 > the model's max sequence length of 1024.
We shall truncate it ourselves. It's imperative if you correct this issue first.
Unsloth: Input IDs of shape torch.Size([1, 14


=== Overall metrics (Expert 2 / LegalBench-Instruct eval) ===
Samples              : 1000
Strict EM (%)        : 70.60
Contains GT (%)      : 72.10
Label Accuracy (%)   : 72.10  (on 871 label-type samples)
ROUGE-L F1 (%)       : 62.88  (on 129 non-label samples)

--- Evaluation complete. ---

--- Starting Expert 2 Evaluation on LegalBench-Instruct (test split) ---
Using existing Test split -> size: 1000
Evaluating on a subset: 1000 samples


  0%|          | 0/1000 [00:00<?, ?it/s]


=== Overall metrics (Expert 2 / LegalBench-Instruct eval) ===
Samples              : 1000
Strict EM (%)        : 42.70
Contains GT (%)      : 63.10
Label Accuracy (%)   : 59.01  (on 871 label-type samples)
ROUGE-L F1 (%)       : 43.21  (on 129 non-label samples)

--- Evaluation complete. ---

=== Delta (FT - BASE) ===
Strict EM Δ (%)      : +27.90
Contains GT Δ (%)    : +9.00
Label Acc Δ (%)      : +13.09 (label_n=871)
ROUGE-L Δ (%)        : +19.66 
Results successfully saved to: /content/drive/Shareddrives/MIRCV Project/llm2/test_results/test_results_lb.json


**Evaluate** best merged models found

In [ ]:
selected_merged_files = ["merged_model_ties_kr03_r32_a640",
                         "merged_model_dare_p03_r32_a640"]

with open(TEST_RESULTS_FILE_ch, "r") as f:
    _res_ch = json.load(f)
BASE_ACC_CH = float(_res_ch["base_model_accuracy_caseHOLD"])
EXPERT1_ACC_CH = float(_res_ch["expert_model_accuracy_caseHOLD"])

with open(TEST_RESULTS_FILE_lb, "r") as f:
    _res_lb = json.load(f)
BASE_LB = _res_lb["base_model_metrics_legalbench"]
EXPERT2_LB = _res_lb["expert2_model_metrics_legalbench"]

MERGED_MANIFEST = os.path.join(BASE_PATH, "merged_adapters_manifest.json")
if os.path.exists(MERGED_MANIFEST):
    with open(MERGED_MANIFEST, "r", encoding="utf-8") as f:
        manifest = json.load(f)
    merged_items = manifest

else:
    # Fallback: scan Drive for merged adapters
    import glob
    merged_dirs = sorted(glob.glob(os.path.join(BASE_PATH, "merged_model*")))
    merged_items = [{"out_dir": d, "mode": None, "keep_ratio": None, "dare_drop_rate": None} for d in merged_dirs]

selected_merged_items = [
    it for it in merged_items
    if adapter_id(it["out_dir"]) in selected_merged_files
]
print(f"Found {len(selected_merged_items)} merged adapters to evaluate.")

Found 2 merged adapters to evaluate.


In [ ]:
PERFORM_TEST_EVALUATION_selected = not os.path.exists(os.path.join(TEST_DIR, "summary_table.csv"))

all_results_selected = []

if PERFORM_TEST_EVALUATION_selected:
    print("\nEvaluating selected merged adapters...")

    test_datasets = {"ch":test_dataset_ch, "lb":test_dataset_lb}
    for it in selected_merged_items:
        meta = {k: it.get(k) for k in ["mode", "keep_ratio", "dare_drop_rate", "out_dir"]}
        res = evaluate_merged_adapter(it["out_dir"], TEST_DIR, meta, test_datasets, using_test_set=True, base_acc_ch=BASE_ACC_CH, base_acc_lb=BASE_LB, exp_acc_ch=EXPERT1_ACC_CH, exp_acc_lb=EXPERT2_LB)
        all_results_selected.append(res)

    df_selected = build_merged_summary_table(
        all_results_selected,
        dataset_lb_dim = len(test_dataset_lb),
        merged_eval_dir=TEST_DIR,
    )
else:
    print(f"Skipping evaluation as results file already exists at: {TEST_DIR}")
    df_selected = pd.read_csv(os.path.join(TEST_DIR, "summary_table.csv"))
    display(df_selected)
    for it in selected_merged_items:
        res_file = eval_cache_path(TEST_DIR, it["out_dir"])
        with open(res_file, "r", encoding="utf-8") as f:
            res = json.load(f)
        all_results_selected.append(res)


Evaluating selected merged adapters...

Evaluating: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_ties_kr03_r32_a640
Meta: {'mode': 'ties', 'keep_ratio': 0.3, 'dare_drop_rate': None}
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Starting evaluation on Test set...


  0%|          | 0/1000 [00:00<?, ?it/s]


--- Starting Expert 2 Evaluation on LegalBench-Instruct (test split) ---
Using existing Test split -> size: 1000
Evaluating on a subset: 1000 samples


  0%|          | 0/1000 [00:00<?, ?it/s]


=== Overall metrics (Expert 2 / LegalBench-Instruct eval) ===
Samples              : 1000
Strict EM (%)        : 66.60
Contains GT (%)      : 67.60
Label Accuracy (%)   : 68.77  (on 871 label-type samples)
ROUGE-L F1 (%)       : 54.11  (on 129 non-label samples)

--- Evaluation complete. ---

Evaluating: /content/drive/Shareddrives/MIRCV Project/llm2/merged_model_dare_p03_r32_a640
Meta: {'mode': 'dare', 'keep_ratio': None, 'dare_drop_rate': 0.3}
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

Starting evaluation on Test set...


...


--- Starting Expert 2 Evaluation on LegalBench-Instruct (test split) ---
Using existing Test split -> size: 1000
Evaluating on a subset: 1000 samples


  0%|          | 0/1000 [00:00<?, ?it/s]


=== Overall metrics (Expert 2 / LegalBench-Instruct eval) ===
Samples              : 1000
Strict EM (%)        : 63.70
Contains GT (%)      : 65.20
Label Accuracy (%)   : 66.25  (on 871 label-type samples)
ROUGE-L F1 (%)       : 48.88  (on 129 non-label samples)

--- Evaluation complete. ---


,adapter,mode,keep_ratio,dare_drop_rate,casehold_acc,casehold_delta_vs_base,legalbench_strict_em,legalbench_contains,legalbench_label_acc,legalbench_rougeL_f1,legalbench_n
1,merged_model_dare_p03_r32_a640,dare,NaN,0.3,69.1,13.4,63.7,65.2,66.245695,48.876584,1000
0,merged_model_ties_kr03_r32_a640,ties,0.3,NaN,48.1,-7.6,66.6,67.6,68.771527,54.109142,1000


Saved summary to: /content/drive/Shareddrives/MIRCV Project/llm2/test_results/summary_table.csv


In [ ]:
# -----------------------------
# Final result flattening / reporting utilities
# -----------------------------
from typing import Any, Dict, Iterable, List, Optional, Tuple
import numpy as np
import pandas as pd
import os, json

def _safe_load_json(path: str) -> Optional[dict]:
    """Load a JSON file if it exists, else return None."""
    if not path or (not os.path.exists(path)):
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_casehold_results(results_file: str) -> Optional[dict]:
    """Schema: {base_model_accuracy_caseHOLD, expert_model_accuracy_caseHOLD, eval_samples_caseHOLD}."""
    data = _safe_load_json(results_file)
    if not data:
        return None
    return {
        "casehold_base_acc": float(data.get("base_model_accuracy_caseHOLD")) if data.get("base_model_accuracy_caseHOLD") is not None else np.nan,
        "casehold_expert_acc": float(data.get("expert_model_accuracy_caseHOLD")) if data.get("expert_model_accuracy_caseHOLD") is not None else np.nan,
        "casehold_n": float(data.get("eval_samples_caseHOLD")) if data.get("eval_samples_caseHOLD") is not None else np.nan,
    }

def load_legalbench_results(results_file: str) -> Optional[dict]:
    """Schema: {base_model_metrics_legalbench, expert2_model_metrics_legalbench, eval_samples_legalbench}."""
    data = _safe_load_json(results_file)
    if not data:
        return None

    base = data.get("base_model_metrics_legalbench") or {}
    expert = data.get("expert2_model_metrics_legalbench") or {}

    def _m(prefix: str, m: dict) -> dict:
        return {
            f"legalbench_{prefix}_strict_em": m.get("strict_em", np.nan),
            f"legalbench_{prefix}_contains": m.get("contains", np.nan),
            f"legalbench_{prefix}_label_acc": m.get("label_acc", np.nan),
            f"legalbench_{prefix}_rougeL_f1": m.get("rougeL_f1", np.nan),
            f"legalbench_{prefix}_n": m.get("n", np.nan),
        }

    out = {}
    out.update(_m("base", base))
    out.update(_m("expert2", expert))
    out["legalbench_eval_n"] = float(data.get("eval_samples_legalbench")) if data.get("eval_samples_legalbench") is not None else np.nan
    return out

def _extract_merged_row(r: dict) -> dict:
    """Extract a single merged-adapter evaluation result into a flat row."""
    meta = r.get("meta", {}) or {}
    out_dir = meta.get("out_dir") or r.get("adapter_dir") or r.get("out_dir")
    adapter = adapter_id(out_dir) if out_dir else meta.get("adapter", "Unknown")

    row = {
        "model_id": adapter,
        "model_type": "merged",
        "merge_mode": meta.get("mode"),
        "keep_ratio": meta.get("keep_ratio"),
        "dare_drop_rate": meta.get("dare_drop_rate"),
        "out_dir": out_dir,
    }

    # CASEHOLD eval (merged on eval_dataset_ch)
    ch = r.get("casehold", {}) or {}
    # unify common field names
    merged_acc = ch.get("merged_acc", ch.get("expert_model_accuracy_caseHOLD", np.nan))
    base_acc = ch.get("base_acc", ch.get("base_model_accuracy_caseHOLD", np.nan))
    row["casehold_eval_acc"] = merged_acc
    row["casehold_eval_base_acc"] = base_acc
    row["casehold_eval_delta_vs_base"] = (merged_acc - base_acc) if (merged_acc is not None and base_acc is not None) else np.nan

    # LEGALBENCH eval (merged on eval_dataset_lb)
    lb = r.get("legalbench", {}) or {}
    lbm = lb.get("merged") or lb.get("expert2_model_metrics_legalbench") or lb.get("metrics") or {}
    row["legalbench_eval_strict_em"] = lbm.get("strict_em", np.nan)
    row["legalbench_eval_contains"] = lbm.get("contains", np.nan)
    row["legalbench_eval_label_acc"] = lbm.get("label_acc", np.nan)
    row["legalbench_eval_rougeL_f1"] = lbm.get("rougeL_f1", np.nan)
    row["legalbench_eval_n"] = lbm.get("n", np.nan)

    return row

Using the results obtained on the test sets we can create a table to resume them, one foreach evaluation type (on Expert 1 or Expert 2).

In [ ]:
def _build_casehold_table(
    *,
    split: str,
    ch: dict,
    merged_results: Iterable[dict],
) -> pd.DataFrame:
    """
    CaseHOLD table: base + expert1 + merged
    Keeps model meta/hparams and ONLY CaseHOLD metrics for the chosen split.
    """
    rows: List[Dict[str, Any]] = []

    # Base
    rows.append({
        "model_id": "base",
        "model_type": "base",
        "ft_target": np.nan,
        "merge_mode": np.nan,
        "keep_ratio": np.nan,
        "dare_drop_rate": np.nan,
        f"casehold_{split}_acc": ch.get("casehold_base_acc", np.nan),
        f"casehold_{split}_n": ch.get("casehold_n", np.nan),
    })

    # Expert1 (fine-tuned on CaseHOLD)
    rows.append({
        "model_id": "expert1",
        "model_type": "fine_tuned",
        "ft_target": "casehold",
        "merge_mode": np.nan,
        "keep_ratio": np.nan,
        "dare_drop_rate": np.nan,
        f"casehold_{split}_acc": ch.get("casehold_expert_acc", np.nan),
        f"casehold_{split}_n": ch.get("casehold_n", np.nan),
    })

    # Merged rows (keep meta/hparams, CaseHOLD-only metric)
    for r in merged_results:
        mr = _extract_merged_row(r)
        rows.append({
            "model_id": mr.get("model_id"),
            "model_type": mr.get("model_type", "merged"),
            "ft_target": np.nan,
            "merge_mode": mr.get("merge_mode", np.nan),
            "keep_ratio": mr.get("keep_ratio", np.nan),
            "dare_drop_rate": mr.get("dare_drop_rate", np.nan),
            f"casehold_{split}_acc": mr.get("casehold_eval_acc", np.nan),   # from extractor
            f"casehold_{split}_n": ch.get("casehold_n", np.nan),            # best effort (same eval size)
        })

    df = pd.DataFrame(rows)

    # column order
    cols = [
        "model_id", "model_type", "ft_target",
        "merge_mode", "keep_ratio", "dare_drop_rate", "out_dir",
        f"casehold_{split}_acc", f"casehold_{split}_n",
    ]
    for c in cols:
        if c not in df.columns:
            df[c] = np.nan
    other_cols = [c for c in df.columns if c not in cols]
    return df[cols + other_cols]


def _build_legalbench_table(
    *,
    split: str,
    lb: dict,
    merged_results: Iterable[dict],
) -> pd.DataFrame:
    """
    LegalBench table: base + expert2 + merged
    Keeps model meta/hparams and ONLY LegalBench metrics for the chosen split.
    """
    rows: List[Dict[str, Any]] = []

    # Base
    rows.append({
        "model_id": "base",
        "model_type": "base",
        "ft_target": np.nan,
        "merge_mode": np.nan,
        "keep_ratio": np.nan,
        "dare_drop_rate": np.nan,
        f"legalbench_{split}_strict_em": lb.get("legalbench_base_strict_em", np.nan),
        f"legalbench_{split}_contains": lb.get("legalbench_base_contains", np.nan),
        f"legalbench_{split}_label_acc": lb.get("legalbench_base_label_acc", np.nan),
        f"legalbench_{split}_rougeL_f1": lb.get("legalbench_base_rougeL_f1", np.nan),
        f"legalbench_{split}_n": lb.get("legalbench_base_n", np.nan),
    })

    # Expert2 (fine-tuned on LegalBench)
    rows.append({
        "model_id": "expert2",
        "model_type": "fine_tuned",
        "ft_target": "legalbench",
        "merge_mode": np.nan,
        "keep_ratio": np.nan,
        "dare_drop_rate": np.nan,
        f"legalbench_{split}_strict_em": lb.get("legalbench_expert2_strict_em", np.nan),
        f"legalbench_{split}_contains": lb.get("legalbench_expert2_contains", np.nan),
        f"legalbench_{split}_label_acc": lb.get("legalbench_expert2_label_acc", np.nan),
        f"legalbench_{split}_rougeL_f1": lb.get("legalbench_expert2_rougeL_f1", np.nan),
        f"legalbench_{split}_n": lb.get("legalbench_expert2_n", np.nan),
    })

    # Merged rows (keep meta/hparams, LegalBench-only metrics)
    for r in merged_results:
        mr = _extract_merged_row(r)
        rows.append({
            "model_id": mr.get("model_id"),
            "model_type": mr.get("model_type", "merged"),
            "ft_target": np.nan,
            "merge_mode": mr.get("merge_mode", np.nan),
            "keep_ratio": mr.get("keep_ratio", np.nan),
            "dare_drop_rate": mr.get("dare_drop_rate", np.nan),
            f"legalbench_{split}_strict_em": mr.get("legalbench_eval_strict_em", np.nan),
            f"legalbench_{split}_contains": mr.get("legalbench_eval_contains", np.nan),
            f"legalbench_{split}_label_acc": mr.get("legalbench_eval_label_acc", np.nan),
            f"legalbench_{split}_rougeL_f1": mr.get("legalbench_eval_rougeL_f1", np.nan),
            f"legalbench_{split}_n": mr.get("legalbench_eval_n", np.nan),
        })

    df = pd.DataFrame(rows)

    cols = [
        "model_id", "model_type", "ft_target",
        "merge_mode", "keep_ratio", "dare_drop_rate", "out_dir",
        f"legalbench_{split}_strict_em",
        f"legalbench_{split}_contains",
        f"legalbench_{split}_label_acc",
        f"legalbench_{split}_rougeL_f1",
        f"legalbench_{split}_n",
    ]
    for c in cols:
        if c not in df.columns:
            df[c] = np.nan
    other_cols = [c for c in df.columns if c not in cols]
    return df[cols + other_cols]

In [ ]:
def flatten_results(
    *,
    dataset: str,  # "casehold" or "legalbench"
    results_json: str,
    merged_results: Iterable[dict],
    out_csv_path: str,
) -> pd.DataFrame:
    """
    Create ONE dataset-specific table (NOT a single unified table):

    - dataset="casehold": base + expert1 + merged, CaseHOLD metrics only
    - dataset="legalbench": base + expert2 + merged, LegalBench metrics only

    The evaluation split is inferred from the JSON paths:
      - if "test" appears in the filename/path -> split="test"
      - else split="eval"

    Saves the produced table to out_csv_path and returns the DataFrame.
    """

    dataset = dataset.strip().lower()
    if dataset not in {"casehold", "legalbench"}:
        raise ValueError(f"dataset must be 'casehold' or 'legalbench', got: {dataset}")

    def _infer_split(*paths: str) -> str:
        joined = " ".join([p.lower() for p in paths if p])
        return "test" if "test" in joined else "eval"

    split = _infer_split(results_json)

    # Load split-specific results (we load both JSONs, but we will use only the relevant one)

    # Build the chosen table using the appropriate builder
    if dataset == "casehold":
        ch = load_casehold_results(results_json) or {}
        df = _build_casehold_table(split=split, ch=ch, merged_results=merged_results)
    else:
        lb = load_legalbench_results(results_json) or {}
        df = _build_legalbench_table(split=split, lb=lb, merged_results=merged_results)

    # Persist the chosen table
    df.to_csv(out_csv_path, index=False)
    return df


In [ ]:
# -----------------------------
# Build the final summary table and save merged-only CSV
# -----------------------------
summary_df_ch = flatten_results(
    dataset="casehold",
    results_json=TEST_RESULTS_FILE_ch,
    merged_results=all_results_selected,
    out_csv_path=os.path.join(TEST_DIR, "test_summary_ch.csv"),
)

summary_df_ch

,model_id,model_type,ft_target,merge_mode,keep_ratio,dare_drop_rate,out_dir,casehold_test_acc,casehold_test_n
0,base,base,NaN,NaN,NaN,NaN,NaN,55.7,1000.0
1,expert1,fine_tuned,casehold,NaN,NaN,NaN,NaN,73.9,1000.0
2,merged_model_ties_kr03_r32_a640,merged,NaN,ties,0.3,NaN,/content/drive/Shareddrives/MIRCV Project/llm2...,48.1,1000.0
3,merged_model_dare_p03_r32_a640,merged,NaN,dare,NaN,0.3,/content/drive/Shareddrives/MIRCV Project/llm2...,69.1,1000.0


In [ ]:
# -----------------------------
# Build the final summary table and save merged-only CSV
# -----------------------------
summary_df_lb = flatten_results(
    dataset="legalbench",
    results_json=TEST_RESULTS_FILE_lb,
    merged_results=all_results_selected,
    out_csv_path=os.path.join(TEST_DIR, "test_summary_lb.csv"),
)

summary_df_lb

,model_id,model_type,ft_target,merge_mode,keep_ratio,dare_drop_rate,out_dir,legalbench_test_strict_em,legalbench_test_contains,legalbench_test_label_acc,legalbench_test_rougeL_f1,legalbench_test_n
0,base,base,NaN,NaN,NaN,NaN,NaN,42.7,63.1,59.012629,43.211098,1000
1,expert2,fine_tuned,legalbench,NaN,NaN,NaN,NaN,70.6,72.1,72.101033,62.875600,1000
2,merged_model_ties_kr03_r32_a640,merged,NaN,ties,0.3,NaN,/content/drive/Shareddrives/MIRCV Project/llm2...,66.6,67.6,68.771527,54.109142,1000
3,merged_model_dare_p03_r32_a640,merged,NaN,dare,NaN,0.3,/content/drive/Shareddrives/MIRCV Project/llm2...,63.7,65.2,66.245695,48.876584,1000


In conclusion, this project demonstrates that while fine-tuning specialized 'expert' models yields excellent results, merging them into a single unified model remains a complex challenge. Individually, our experts significantly outperformed the base Llama-3 model, with Expert 1 improving CaseHOLD accuracy by over 18% and Expert 2 mastering LegalBench instructions.

 However, combining these capabilities was reavealed a challenge: **the merged models** (using TIES and DARE) consistently performed worse than the single experts (as expected), but in some configurations could outperfrom the base model, revealing the usefulness of these methods. In our tests we can observe how the TIES method struggled to retain the performances on the CaseHOLD dataset, and the DARE method is able to fit as a good trade off bewtween expert and base models.

 This suggests that while LoRA adapters are proved efficient for training, they are fragile during merging and should be handled accordingly.

The performance degradation observed in our merged models can be explained by key technical factors:.

- There could be an incompatibility between sprasifying methods (like DARE or TIES which drop or trim weights) and LoRA. TIES relies on 'trimming' redundant parameters, but since LoRA adapters already encode knowledge in sparse, low-rank updates, this aggressive pruning can remove critical task-specific information, leading to the sharp drop in accuracy.
- TIES-Merging often underperforms on decoder-only architectures (like Llama-3) compared to encoder-decoder models, further exacerbating the parameter conflict.

### Qualitative evaluation

To complement our automated metrics, we introduce the `qualitative_eval` function. This utility allows for a manual inspection of the model's behavior by displaying the full input context alongside the generated response and the ground truth. By forcing the model to explain its reasoning before answering, this function serves as a critical window into the model's decision-making process, ensuring the final merged architecture is both accurate and interpretable.

In [ ]:
from typing import Dict, Any

def qualitative_eval(
    model,
    tokenizer,
    datasets: Dict[str, Any],
    *,
    n_samples: int = 3,
    split: str = "mixed",
    seed: int = 42,
    max_new_tokens: int = 256,
    temperature: float = 0.3,
    top_p: float = 0.9,
    do_sample: bool = True,
) -> None:

    random.seed(seed)
    device = next(model.parameters()).device

    # Prompt Engineering functions

    def _get_casehold_prompt(ex: dict) -> str:
        label_map = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}
        ctx = clean_legal_text(ex["context"])
        opts = [clean_legal_text(opt) for opt in ex["endings"]]
        formatted_opts = "\n".join([f"{label_map[i]}) {opt}" for i, opt in enumerate(opts)])

        # Prompt asking for reasoning
        messages = [
            {"role": "system", "content": "You are a legal expert. Analyze the case context and options carefully."},
            {"role": "user", "content": f"CONTEXT:\n{ctx}\n\nOPTIONS:\n{formatted_opts}\n\nTASK:\n1. Provide a concise explanation of your legal reasoning.\n2. Identify the correct holding (A, B, C, D, or E)."}
        ]
        return tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    def _extract_legalbench(ex: dict):
        user_prompt = clean_legal_text(ex["inputs"])
        gt = str(ex["answer"])

        system_prompt = "You are a legal expert. Perform the requested task and demonstrate your reasoning."

        # "Reasoning First" Prompt
        enhanced_user_prompt = f"""{user_prompt}

          ### INSTRUCTIONS:
          Step 1: Explain your reasoning in 1-2 sentences.
          Step 2: Provide the final answer."""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": enhanced_user_prompt},
        ]

        prompt_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs_dict = tokenizer(prompt_text, return_tensors="pt").to(device)

        # We also return the clean 'user_prompt' to print it legibly
        readable_question = f"{user_prompt}"

        return prompt_text, inputs_dict, gt, readable_question

    # EXECUTION
    if split == "mixed":
      buckets = ["ch", "lb"]
    elif split in {"ch", "lb"}:
      buckets = [split]

    print("\n" + "="*90)
    print(f" FULL VISUAL EVALUATION ")
    print("="*90 + "\n")

    sample_id = 1

    for b in buckets:
        ds = datasets.get(b)
        if not ds: continue

        print(f"\n --- Dataset: {b} ---\n")

        # Generate N samples
        for _ in range(n_samples):
            ex = None

            # FILTER LOGIC FOR LEGALBENCH (Looks for non-binary examples)
            # This ensures that when we run the evaluation,
            # we can see complex tasks (like text extraction or explanation) rather than simple binary choices, which are less useful for qualitative analysis.
            if b == "lb":
                attempts = 0
                while not ex and attempts < 100:
                    rand_idx = random.randint(0, len(ds)-1)
                    candidate = dict(ds[rand_idx])
                    ans_text = str(candidate["answer"]).strip().lower()
                    # Filter out Yes/No to see more interesting questions
                    if ans_text not in ["yes", "no", "true", "false"] and len(ans_text) > 3:
                        ex = candidate
                        idx_used = rand_idx
                    attempts += 1
                if not ex: ex = dict(ds[random.randint(0, len(ds)-1)]); idx_used = -1
            else:
                idx_used = random.randint(0, len(ds)-1)
                ex = dict(ds[idx_used])

            # INPUT PREPARATION
            if b == "ch":
                prompt_text = _get_casehold_prompt(ex)
                inputs_dict = tokenizer(prompt_text, return_tensors="pt").to(device)
                input_ids = inputs_dict['input_ids']

                gt_label = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}[int(ex["label"])]
                gt_text = f"Option {gt_label}"

                # For CaseHOLD, we show the context and options as 'Visible Input'
                readable_input = f"CONTEXT (Extract):\n{ex['context'][:400]}...\n\nOPTIONS:\n" + "\n".join([f"- {opt}" for opt in ex["endings"]])
            else:
                prompt_text, inputs_dict, gt_text, readable_input = _extract_legalbench(ex)
                input_ids = inputs_dict['input_ids']

            # GENERATION
            outputs = model.generate(
                input_ids,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                do_sample=do_sample,
                top_p=top_p,
                pad_token_id=tokenizer.eos_token_id
            )

            response = tokenizer.decode(outputs[0][input_ids.shape[1]:], skip_special_tokens=True).strip()

            # --- PRINT RESULTS ---
            print(f" [Sample #{sample_id}] Index: {idx_used} | Dataset: {b}")
            print("-" * 90)

            # 1. SHOW QUESTION/INPUT (No special tokens, just readable text)
            print(f"INPUT (QUESTION & CONTEXT):\n")
            print(readable_input)

            print("-" * 40)

            # 2. SHOW MODEL RESPONSE
            print(f"MODEL OUTPUT (Reasoning + Answer):\n")
            print(response)

            print("-" * 40)

            # 3. SHOW GROUND TRUTH
            print(f"GROUND TRUTH:\n{gt_text}")

            print("="*90 + "\n")
            sample_id += 1

In [ ]:
BEST_MERGED_MODEL_PATH = BASE_PATH + "merged_model_dare_p03_r32_a640"
model, tokenizer = load_base_with_lora_adapter(BEST_MERGED_MODEL_PATH)
model.eval()

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2026.1.4 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
       

In [ ]:
qualitative_eval(
    model=model,
    tokenizer=tokenizer,
    datasets={"ch": test_dataset_ch, "lb": test_dataset_lb},
    n_samples=2,
    split="mixed",   # or "ch" or "lb"
    seed=42,
    max_new_tokens=256,
    do_sample=False,  # deterministic
)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



 FULL VISUAL EVALUATION 


 --- Dataset: ch ---

 [Sample #1] Index: 654 | Dataset: ch
------------------------------------------------------------------------------------------
INPUT (QUESTION & CONTEXT):

CONTEXT (Extract):
In re Gearheart, No. 07-70232, 2007 WL 4463342 (Bankr.E.D.Ky. Dec. 14, 2007); In re Logan, No. 07-70212, 2007 WL 4414784 (Bankr.E.D.Ky. Dec. 14, 2007); In re Fuller, No. 07-81703, 2007 WL 3244113 (Bankr.M.D.N.C. Nov. 2, 2007); In re Oliveira, 378 B.R. 789 (Bankr.E.D.Tex.2007); Herrin v. Green Tree-AL, LLC, 376 B.R. 316 (Bankr.S.D.Ala.2007), aff'g In re Herrin, No. 06-12249-WSS-13, 2007 WL 1975573 ...

OPTIONS:
- holding that the antimodification provision of 11 usc  1322b is applicable to a mobile home irrespective of whether the home is attached to the real property on which it sits
- holding that the definition of debtors principal residence in 11 usc  10113aa protects holder of secured claim in a mobile home that is debtors principal residence from modificatio

### References


The following research papers and articles were used as theoretical and methodological references throughout this project:

1. **LoRA: Low-Rank Adaptation of Large Language Models, 2021**:
  
      Link: https://arxiv.org/pdf/2106.09685v1/1000

2. **Language Models are Super Mario: Absorbing abilities from Homolongous Models as a Free Lunch, 2024**

      Link: https://arxiv.org/pdf/2311.03099

3. **TIES-MERGING: Resolving Interference when merging methods, 2023**

      Link: https://arxiv.org/pdf/2306.01708
